In [2]:
! pip install geopandas
! pip install pandas
! pip install pygeos
! pip install boto
! pip install s3fs
! pip install shapely
! pip install gcsfs
!pip install contextily
!pip install matplotlib-scalebar
!pip install sklearn
!pip install zipfile
!pip install random
import random
import numpy as np
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
import gzip
import time
from itertools import groupby
import geopandas as gpd
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.ticker as ticker
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 43.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be gr

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_4094033/985655068.py:21: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).

In [ ]:
def addGeometryIdToDataFrame(df, gdf, xcol, ycol, idColumn="geometry", df_geom='epsg:4326'): 
    gdf.crs = {'init': 'epsg:4326'}
    gdf_data = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[xcol], df[ycol]))
    gdf_data.crs = {'init': df_geom}
    joined = gpd.sjoin(gdf_data.to_crs('epsg:26910'), gdf.to_crs('epsg:26910'))
    gdf_data = gdf_data.merge(joined['zip'], left_index=True, right_index=True, how="left")
    gdf_data.rename(columns={'zip': idColumn}, inplace=True)
    df = pd.DataFrame(gdf_data.drop(columns='geometry'))
    # df.drop(columns=[xcol, ycol], inplace=True)
    
    return df.loc[~df.index.duplicated(keep='first'), :]

In [1]:
#Read
BGs = gpd.read_file('inputs/geo_export_9539336a-a70d-42a4-93a6-0df79b218c82.shp')

Dir_sce = 'sfbay-baseline2018-30pct-20230710'
Dir_sce_TR = 'sfbay-tr-30pct-20230716'
year = '2018'
ite = '10'

Dir_beam = 'gs://beam-core-outputs/'+Dir_sce+'/beam/year-'+year+'-iteration-'+ite+'/ITERS/it.0'
Dir_act = 'gs://beam-core-outputs/'+Dir_sce+'/activitysim/year-'+year+'-iteration-'+ite
Dir_beam_TR = 'gs://beam-core-outputs/'+Dir_sce_TR+'/beam/year-'+year+'-iteration-'+ite+'/ITERS/it.0'
Dir_act_TR = 'gs://beam-core-outputs/'+Dir_sce_TR+'/activitysim/year-'+year+'-iteration-'+ite

Persons = pd.read_csv(Dir_act+'/persons.csv.gz', nrows = None)
Households = pd.read_csv(Dir_act+'/households.csv.gz', nrows = None)
Plans = pd.read_csv(Dir_beam+'/0.plans.csv.gz', nrows = None)

person_table_path = 'outputs/person_databaseTR.csv'
Scale = 0.3

NameError: name 'gpd' is not defined

# ***PERSON TABLE ANALYSIS***

In [ ]:
df = pd.read_csv(person_table_path)

df = df[df['Vehicles Used Baseline'].astype(str) != '[]']
df = df[~((df['Chosen Mode Baseline'] == 'walk') & (df['Trip Length Baseline'] > 5000))]


race_dict = {
    1: "White alone",
    2: "Black or African American alone",
    3: "American Indian alone",
    4: "Alaska Native alone",
    5: "American Indian and Alaska Native tribes specified; or American Indian or Alaska Native, not specified and no other races",
    6: "Asian alone",
    7: "Native Hawaiian and Other Pacific Islander alone",
    8: "Some Other Race alone",
    9: "Two or More Races"
}
def addGeometryIdToDataFrame(df, gdf, xcol, ycol, idColumn="geometry", df_geom='epsg:32610'): 
    gdf.set_crs(epsg = "3310", inplace = True)
    gdf_data = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[xcol], df[ycol]))
    gdf_data.set_crs(epsg = "32610", inplace = True)
    joined = gpd.sjoin(gdf_data.to_crs('epsg:26910'), gdf.to_crs('epsg:26910'))
    gdf_data = gdf_data.merge(joined['ZCTA'], left_index=True, right_index=True, how="left")
    gdf_data.rename(columns={'ZCTA': idColumn}, inplace=True)
    df = pd.DataFrame(gdf_data.drop(columns='geometry'))
#     df.drop(columns=[xcol, ycol], inplace=True)
    return df.loc[~df.index.duplicated(keep='first'), :]


print(len(df))

zipcode = gpd.read_file('inputs/ZCTA2010.shp')
print(len(df))

df = addGeometryIdToDataFrame(df, zipcode, 'X Activity From TR', 'Y Activity From TR', 'ZIP Departure TR')
print(len(df))

df = addGeometryIdToDataFrame(df, zipcode, 'X Activity To TR', 'Y Activity To TR', 'ZIP Arrival TR')
print(len(df))

df = addGeometryIdToDataFrame(df, zipcode, 'X Activity From Baseline', 'Y Activity From Baseline', 'ZIP Departure Baseline')
print(len(df))

df = addGeometryIdToDataFrame(df, zipcode, 'X Activity To Baseline', 'Y Activity To Baseline', 'ZIP Arrival Baseline')
print(len(df))


#Add Switch From Column SF - Van Ness
switch_type = []
for agencies_baseline, project in zip(df['Bus agencies Used Baseline'],df['Project Tried']):
    if agencies_baseline == '[]':
        switch_type.append('Switch from another mode')
    elif project == 'SF - Central Subway' and 'SF' in agencies_baseline:
        switch_type.append('Switch from same transit agency')
    elif project == 'AC - 1TEMPO' and 'AC' in agencies_baseline:
        switch_type.append('Switch from same transit agency')
    elif project == 'CA - Electrification Project' and 'ca' in agencies_baseline:
        switch_type.append('Switch from same transit agency')
    elif project == 'SF - Van Ness' and 'SF' in agencies_baseline:
        switch_type.append('Switch from same transit agency')
    elif project == 'BA - Extension and Core project' and 'BA' in agencies_baseline:
        switch_type.append('Switch from same transit agency')
    else:
        switch_type.append('Switch from another transit agency')

df['Switch From'] = switch_type
print(len(df))

persons = pd.read_csv('gs://beam-core-outputs/sfbay-baseline2018-30pct-20230710/activitysim/final_persons.csv')
households = pd.read_csv('gs://beam-core-outputs/sfbay-baseline2018-30pct-20230710/activitysim/final_households.csv')

person_to_pincome = dict(zip(persons.person_id, persons.earning))
person_to_race = dict(zip(persons.person_id, persons.race_id))

person_to_hs = dict(zip(persons.person_id, persons.household_id))
hs_to_income = dict(zip(households.household_id, households.income_in_thousands*1000))
person_to_income = {key: hs_to_income[person_to_hs[key]] for key in person_to_hs}


df['HS Income'] = df['Person'].map(person_to_income) 
df['Person Income'] = df['Person'].map(person_to_pincome) 
df['Person Race ID'] = df['Person'].map(person_to_race) 
df['Person Race'] = df['Person Race ID'].map(race_dict) 

df.to_csv('outputs/person_databaseTR-upgraded.csv', index = False)


# **Ridership**

In [ ]:
import ast


dict_project = {
    'AC - 1TEMPO':'1T',
    'BA - Extension and Core project':'BA',
    'CA - Electrification Project':'CA',
    'SF - Central Subway':'T',
    'SF - Van Ness': '49'
}

veh_to_route_bs = pd.read_csv('outputs/routetovehicledictBaseline2018.csv')
veh_to_route_bs = veh_to_route_bs.set_index('trip_id')['route_id'].to_dict()
veh_to_route_TR = pd.read_csv('outputs/routetovehicledictFuture2018.csv')
veh_to_route_TR = veh_to_route_TR.set_index('trip_id')['route_id'].to_dict()

from_indexes = [

                '1', '49', 'K/T', 'J', 'L', 'M', 'N', 
                'BA - Blue', 'BA - Green', 'BA - Orange', 'BA - Red', 
                'BA - Yellow', 'Car', 'CarPool','TNC', 'TNC-Pool','Walk', 'Bike', 
    'CA', 'AC - others', 'SF - others','AM','AY','CC','CE','CM',
    'DE','EM','FF','GG','HF','MA','PE','RV', 'SB',
    'SC','SL','SM','SO','SR','ST','TD','UC','VC','VN','VT','WC','WH',
                'Other']
to_indexes_dict = {'CA':['CA:12867','CA:12868','CA:12869'], '1T':'AC:1-142', '49':'SF:12327', 'T':'SF:12476', 
              #'J', 'L', 'M', 'N', 
              'BA Blue':'BA:11', 
              'BA Green':'BA:5', 'BA Orange':'BA:3', 'BA Red':'BA:7', 'BA Yellow':'BA:1'}
to_indexes = ['CA', '1T', '49', 'T', 
              #'J', 'L', 'M', 'N', 
              'BA Blue', 
              'BA Green', 'BA Orange', 'BA Red', 'BA Yellow']
FT3 = pd.DataFrame(0, index=from_indexes, columns=to_indexes)

for veh_bs, veh_tr, project, mode_base in zip(list(df['Vehicles Used Baseline']), list(df['Vehicles Used TR']), list(df['Project Tried']), list(df['Chosen Mode Baseline'])):
    # routes_bs = veh_to_route_bs[veh_bs]
    # routes_TR = veh_to_route_TR[veh_tr]
    # veh_bs = ','.join(veh_bs)
    # veh_tr = ','.join(veh_tr)
    found = 0
    
    fixed_string = veh_bs.replace(" ", ", ")
    veh_bs = ast.literal_eval(fixed_string)
    fixed_string = veh_tr.replace(" ", ", ")
    veh_tr = ast.literal_eval(fixed_string)

    routes_bs = [veh_to_route_bs.get(key, '0') for key in veh_bs]
    routes_TR = [veh_to_route_TR.get(key, '0') for key in veh_tr]
    BARTS = []
    if project != 'BA - Extension and Core project' :
        BARTS.append(dict_project[project])
    else:
        
        if 'BA:1' in routes_TR or 'BA:2' in routes_TR :
            BARTS.append('BA Yellow')
        if 'BA:3' in routes_TR or 'BA:4' in routes_TR :
            BARTS.append('BA Orange')
        if 'BA:5' in routes_TR or 'BA:6' in routes_TR :
            BARTS.append('BA Green')
        if 'BA:7' in routes_TR or 'BA:8' in routes_TR :
            BARTS.append('BA Red')
        if 'BA:11' in routes_TR or 'BA:12' in routes_TR :
            BARTS.append('BA Blue')
            
    for BAR in BARTS:
        
        if BAR == '1T':
            if 'AC:1-142' in routes_bs:
                FT3.loc['1',BAR] += 1
                found+=1
            if any('AC:' in s for s in routes_bs) and not any('AC:' in s for s in [item for item in routes_TR if item != 'AC:1T-142']) and not any('AC:1-142' in s for s in routes_bs) :
                FT3.loc['AC - others',BAR] += 1
                found+=1
        if BAR == 'CA':
            if any('CA:' in s for s in routes_bs) :
                FT3.loc['CA',BAR] += 1
                found+=1
        if BAR == 'BA Yellow':
            if 'BA:1' in routes_bs:
                FT3.loc['BA - Yellow',BAR] += 1
                found+=1
        if BAR == 'BA Orange':
            if 'BA:3' in routes_bs:
                FT3.loc['BA - Orange',BAR] += 1
                found+=1
        if BAR == 'BA Green':
            if 'BA:5' in routes_bs:
                FT3.loc['BA - Green',BAR] += 1
                found+=1
        if BAR == 'BA Red':
            if 'BA:7' in routes_bs:
                FT3.loc['BA - Red',BAR] += 1
                found+=1
        if BAR == 'BA Blue':
            if 'BA:11' in routes_bs:
                FT3.loc['BA - Blue',BAR] += 1
                found+=1
        if BAR == 'T':
            if 'SF:12476' in routes_bs:
                FT3.loc['K/T',BAR] += 1
                found+=1
            if any('SF:' in s for s in routes_bs) and not any('SF:' in s for s in [item for item in routes_TR if item != 'SF:1001']) and not any('SF:12327' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12476' in s for s in routes_bs) and not any('SF:12477' in s for s in routes_bs) and not any('SF:12478' in s for s in routes_bs) and not any('SF:12479' in s for s in routes_bs):
                FT3.loc['SF - others',BAR] += 1
                found+=1
        if BAR == '49':
            if 'SF:12327' in routes_bs:
                FT3.loc['49',BAR] += 1
                found+=1
            if any('SF:' in s for s in routes_bs) and not any('SF:' in s for s in [item for item in routes_TR if item != 'SF:18608']) and not any('SF:12327' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12476' in s for s in routes_bs) and not any('SF:12477' in s for s in routes_bs) and not any('SF:12478' in s for s in routes_bs) and not any('SF:12479' in s for s in routes_bs):
                FT3.loc['SF - others',BAR] += 1
                found+=1
        

        # if any('AC:' in s for s in routes_bs):
        #     FT.loc['AC',BAR] += 1
        # if any('SF:' in s for s in routes_bs):
        #     FT.loc['SF',BAR] += 1
        # if any('BA:' in s for s in routes_bs):
        #     FT.loc['BA',BAR] += 1
            
        if 'AC:1-142' in routes_bs and 'AC:1T-142' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['1',BAR] += 1
            found+=1
        if 'SF:12327' in routes_bs and 'SF:18608' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['49',BAR] += 1
            found+=1
        if 'SF:12476' in routes_bs and 'SF:1001' not in routes_TR and 'SF:1002' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['K/T',BAR] += 1
            found+=1
        if 'SF:12475' in routes_bs and 'SF:12375' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['J',BAR] += 1
            found+=1
        if 'SF:12477' in routes_bs and 'SF:12377' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['L',BAR] += 1
            found+=1
        if 'SF:12478' in routes_bs and 'SF:12378' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['M',BAR] += 1
            found+=1
        if 'SF:12479' in routes_bs and 'SF:12379' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['N',BAR] += 1
            found+=1
        if 'BA:1' in routes_bs and 'BA:1' not in routes_TR and 'BA:2' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['BA - Yellow',BAR] += 1
            found+=1
        if 'BA:3' in routes_bs and 'BA:3' not in routes_TR and 'BA:4' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['BA - Orange',BAR] += 1
            found+=1
        if 'BA:5' in routes_bs and 'BA:5' not in routes_TR and 'BA:6' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['BA - Green',BAR] += 1
            found+=1
        if 'BA:7' in routes_bs and 'BA:7' not in routes_TR and 'BA:8' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['BA - Red',BAR] += 1
            found+=1
        if 'BA:11' in routes_bs and 'BA:11' not in routes_TR and 'BA:12' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['BA - Blue',BAR] += 1
            found+=1
            
        if mode_base =='walk' :
            FT3.loc['Walk',BAR] += 1
            found+=1
        if mode_base =='ride_hail' :
            FT3.loc['TNC',BAR] += 1
            found+=1
        if mode_base =='ride_hail_pooled' :
            FT3.loc['TNC-Pool',BAR] += 1
            found+=1
        if mode_base =='car' :
            FT3.loc['Car',BAR] += 1
            found+=1
        if mode_base =='car_hov2' :
            FT3.loc['CarPool',BAR] += 1
            found+=1
        if mode_base =='car_hov3' :
            FT3.loc['CarPool',BAR] += 1
            found+=1
        if mode_base =='bike' :
            FT3.loc['Bike',BAR] += 1
            found+=1

            
        if any('AC:' in s for s in routes_bs) and not any('AC:' in s for s in routes_TR) and not any('AC:1-142' in s for s in routes_bs)  and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['AC - others',BAR] += 1
            found+=1
        if any('SF:' in s for s in routes_bs) and not any('SF:' in s for s in routes_TR) and not any('SF:12327' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12476' in s for s in routes_bs) and not any('SF:12477' in s for s in routes_bs) and not any('SF:12478' in s for s in routes_bs) and not any('SF:12479' in s for s in routes_bs) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['SF - others',BAR] += 1
            found+=1
        # if any('BA:' in s for s in routes_bs) and not any('BA:' in s for s in routes_TR):
        #     FT.loc['BA',BAR] += 1
        #     found+=1
        
        if any('CA:' in s for s in routes_bs) and not any('CA:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['CA',BAR] += 1
            found+=1
        if any('AM:' in s for s in routes_bs) and not any('AM:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['AM',BAR] += 1
            found+=1
        if any('AY:' in s for s in routes_bs) and not any('AY:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['AY',BAR] += 1
            found+=1
        if any('CC:' in s for s in routes_bs) and not any('CC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['CC',BAR] += 1
            found+=1
        if any('CE:' in s for s in routes_bs) and not any('CE:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['CE',BAR] += 1
            found+=1
        if any('CM:' in s for s in routes_bs) and not any('CM:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['CM',BAR] += 1
            found+=1
        if any('DE:' in s for s in routes_bs) and not any('DE:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['DE',BAR] += 1
            found+=1
        if any('EM:' in s for s in routes_bs) and not any('EM:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['EM',BAR] += 1
            found+=1
        if any('GG:' in s for s in routes_bs) and not any('GG:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['GG',BAR] += 1
            found+=1
        if any('HF:' in s for s in routes_bs) and not any('HF:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['HF',BAR] += 1
            found+=1
        if any('MA:' in s for s in routes_bs) and not any('MA:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['MA',BAR] += 1
            found+=1
        if any('PE:' in s for s in routes_bs) and not any('PE:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['PE',BAR] += 1
            found+=1
        if any('RV:' in s for s in routes_bs) and not any('RV:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['RV',BAR] += 1
            found+=1
        if any('SB:' in s for s in routes_bs) and not any('SB:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['SB',BAR] += 1
            found+=1
        if any('SC:' in s for s in routes_bs) and not any('SC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['SC',BAR] += 1
            found+=1
        if any('SL:' in s for s in routes_bs) and not any('SL:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['SL',BAR] += 1
            found+=1
        if any('SM:' in s for s in routes_bs) and not any('SM:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['SM',BAR] += 1
            found+=1
        if any('SO:' in s for s in routes_bs) and not any('SO:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['SO',BAR] += 1
            found+=1
        if any('SR:' in s for s in routes_bs) and not any('SR:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['SR',BAR] += 1
            found+=1
        if any('ST:' in s for s in routes_bs) and not any('ST:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['ST',BAR] += 1
            found+=1
        if any('TD:' in s for s in routes_bs) and not any('TD:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['TD',BAR] += 1
            found+=1
        if any('UC:' in s for s in routes_bs) and not any('UC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['UC',BAR] += 1
            found+=1
        if any('VC:' in s for s in routes_bs) and not any('VC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['VC',BAR] += 1
            found+=1
        if any('VN:' in s for s in routes_bs) and not any('VN:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['VN',BAR] += 1
            found+=1
        if any('VT:' in s for s in routes_bs) and not any('VT:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['VT',BAR] += 1
            found+=1
        if any('WC:' in s for s in routes_bs) and not any('WC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['WC',BAR] += 1
            found+=1
        if any('WH:' in s for s in routes_bs) and not any('WH:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT3.loc['WH',BAR] += 1
            found+=1    
            
            
            
            
        if found == 0:
            FT3.loc['Other',BAR] += 1  
FT3.loc['Total'] = [FT3['CA'].sum(),FT3['1T'].sum(),FT3['49'].sum(),FT3['T'].sum(),FT3['BA Blue'].sum(),FT3['BA Green'].sum(),FT3['BA Orange'].sum(),FT3['BA Red'].sum(),FT3['BA Yellow'].sum()]

FT3 = (FT3/Scale).astype(int)

In [ ]:
FT3/FT3.loc['Total']*100


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def add_row_name_to_bars(ax, bottoms, values, column_names, row_name, threshold):
    new_bottoms = [bottom + value for bottom, value in zip(bottoms, values)]
    for i, (bottom, new_bottom) in enumerate(zip(bottoms, new_bottoms)):
        mid_point = (bottom + new_bottom) / 2
        if (new_bottom - bottom) > threshold:  # Only add text if there's enough space
            ax.text(i, mid_point, row_name, ha='center', va='center', color='black', fontsize=15)
    return new_bottoms

# Sample data preparation
# [Your data preparation code here, e.g., loading the df3 DataFrame]
df3 = FT3
df3 = df3.drop("Total", axis=0)
# Color mapping for the first plot
color_mapping1= {
    '1': '#A8D5BA',        # Light Green
    'AC - others': '#A8D5BA',
    '49': '#F2B5B5',      # Light Red
    'K/T': '#F2B5B5',
    'J': '#F2B5B5',
    'L': '#F2B5B5',
    'M': '#F2B5B5',
    'N': '#F2B5B5',
    'SF - others': '#F2B5B5',
    'BART': '#A9AEEF',    # Light Blue
    'BA - Blue': '#A9AEEF',
    'BA - Green': '#A9AEEF',  # Pastel Green
    'BA - Orange': '#A9AEEF',  # Pastel Orange
    'BA - Red': '#A9AEEF',     # Pastel Red
    'BA - Yellow': '#A9AEEF',  # Pastel Yellow
    'Car': '#D7CCC8',         # Soft Red
    'CarPool': '#D7CCC8',     # Light Grey
    'TNC': '#D7CCC8',
    'TNC-Pool': '#D7CCC8',
    'Walk': '#D7CCC8',
    'Bike': '#D7CCC8',
    'CA': '#80CBC4',          # Soft Blue
    'AM': '#CEA2E1',         # Soft Purple
    'AY': '#CEA2E1',
    'CC': '#CEA2E1',
    'CE': '#CEA2E1',
    'CM': '#CEA2E1',
    'DE': '#CEA2E1',
    'EM': '#CEA2E1',
    'FF': '#CEA2E1',
    'GG': '#CEA2E1',
    'HF': '#CEA2E1',
    'MA': '#CEA2E1',
    'PE': '#CEA2E1',
    'RV': '#CEA2E1',
    'SB': '#CEA2E1',
    'SC': '#CEA2E1',
    'SL': '#CEA2E1',
    'SM': '#CEA2E1',
    'SO': '#CEA2E1',
    'SR': '#CEA2E1',
    'ST': '#CEA2E1',
    'TD': '#CEA2E1',
    'UC': '#CEA2E1',
    'VC': '#CEA2E1',
    'VN': '#CEA2E1',
    'VT': '#CEA2E1',
    'WC': '#CEA2E1',
    'WH': '#CEA2E1',
    'Other': '#F5F5F5'        # Light Grey
}
color_mapping2 = {
    '1': '#F5F5F5',  # muted dark green
    'AC - others': '#F5F5F5',
    '49': '#F5F5F5',  # muted dark red
    'K/T': '#F5F5F5',
    'J': '#F5F5F5',
    'L': '#F5F5F5',
    'M': '#F5F5F5',
    'N': '#F5F5F5',
    'SF - others': '#F5F5F5',
    'BART': '#F5F5F5',
    'BA - Blue': '#BBDEFB',  # muted blue
    'BA - Green': '#C8E6C9',  # muted green
    'BA - Orange': '#FFE0B2',  # muted orange
    'BA - Red': '#FFCDD2',  # muted red
    'BA - Yellow': '#FFF9C4',  # muted yellow
    'Car': '#F5F5F5',  # dark red
    'CarPool': '#F5F5F5',  # grey
    'TNC': '#F5F5F5',
    'TNC-Pool': '#F5F5F5',
    'Walk': '#F5F5F5',
    'Bike': '#F5F5F5',
    'CA': '#F5F5F5',  # muted dark blue
    'AM': '#F5F5F5',  # muted dark purple
    'AY': '#F5F5F5',
    'CC': '#F5F5F5',
    'CE': '#F5F5F5',
    'CM': '#F5F5F5',
    'DE': '#F5F5F5',
    'EM': '#F5F5F5',
    'FF': '#F5F5F5',
    'GG': '#F5F5F5',
    'HF': '#F5F5F5',
    'MA': '#F5F5F5',
    'PE': '#F5F5F5',
    'RV': '#F5F5F5',
    'SB': '#F5F5F5',
    'SC': '#F5F5F5',
    'SL': '#F5F5F5',
    'SM': '#F5F5F5',
    'SO': '#F5F5F5',
    'SR': '#F5F5F5',
    'ST': '#F5F5F5',
    'TD': '#F5F5F5',
    'UC': '#F5F5F5',
    'VC': '#F5F5F5',
    'VN': '#F5F5F5',
    'VT': '#F5F5F5',
    'WC': '#F5F5F5',
    'WH': '#F5F5F5',
    'Other': '#F5F5F5'  # grey
}
desired_order2 = [
    'AC - others', '1', 
    'SF - others', '49', 
    'K/T', 'J', 'L', 'M', 'N', 
    'BA - Blue', 'BA - Green', 'BA - Orange', 'BA - Red', 'BA - Yellow', 
    'Car', 'CarPool', 'TNC', 'TNC-Pool', 'Walk', 'Bike', 
    'CA',
    'AM', 'AY', 'CC', 'CE', 'CM', 'DE', 'EM', 'FF', 'GG', 'HF', 'MA', 'PE', 'RV', 'SB', 'SC', 
    'SL', 'SM', 'SO', 'SR', 'ST', 'TD', 'UC', 'VC', 'VN', 'VT', 'WC', 'WH',
    'Other'
]
desired_order1 = [
    'AC - others', '1', 
    'SF - others', '49', 
    'K/T', 'J', 'L', 'M', 'N', 
    'BA - Blue', 'BA - Green', 'BA - Orange', 'BA - Red', 'BA - Yellow', 
    'Car', 'CarPool', 'TNC', 'TNC-Pool', 'Walk', 'Bike', 
    'CA',
    'AM', 'AY', 'CC', 'CE', 'CM', 'DE', 'EM', 'FF', 'GG', 'HF', 'MA', 'PE', 'RV', 'SB', 'SC', 
    'SL', 'SM', 'SO', 'SR', 'ST', 'TD', 'UC', 'VC', 'VN', 'VT', 'WC', 'WH',
    'Other'
]
# Assuming you have df3 and color_mapping set as before

# Aggregate BA - type rows into BART
# ba_rows = ['BA - Blue', 'BA - Green', 'BA - Orange', 'BA - Red', 'BA - Yellow']
# df3.loc['BART'] = df3.loc[ba_rows].sum()

# Filter and reorder the rows for both plots
df_first_plot = df3.loc[desired_order1,['CA', '1T','49', 'T']]
df_second_plot = df3.loc[desired_order2,['BA Blue', 'BA Green', 'BA Orange', 'BA Red', 'BA Yellow']]

# Create subplots with 1 row and 2 columns
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

threshold1 = 2 * 220  # This is just an example value; adjust as needed
# Plotting the first graph
bottoms = [0] * len(df_first_plot.columns)
for row_name in df_first_plot.index:

    values = df_first_plot.loc[row_name]
    axs[0].bar(df_first_plot.columns, values, bottom=bottoms, color=color_mapping1[row_name], edgecolor='white')
    bottoms = add_row_name_to_bars(axs[0], bottoms, values, df_first_plot.columns, row_name, threshold1)


threshold2 = 2 * 2000  # This is just an example value; adjust as needed
# Plotting the second graph
bottoms = [0] * len(df_second_plot.columns)
for row_name in df_second_plot.index:

    values = df_second_plot.loc[row_name]
    axs[1].bar(df_second_plot.columns, values, bottom=bottoms, color=color_mapping2[row_name], edgecolor='white')
    bottoms = add_row_name_to_bars(axs[1], bottoms, values, df_second_plot.columns, row_name, threshold2)


# axs[1].set_ylabel('Share')
# axs[1].set_title('Stacked Bar Chart - Second Set of Categories')

# Add % to the y-axis labels for the first plot
# y_ticks_0 = axs[0].get_yticks()
# axs[0].set_yticklabels(['{:.0f}%'.format(y) for y in y_ticks_0])

# # Add % to the y-axis labels for the second plot
# y_ticks_1 = axs[1].get_yticks()
# axs[1].set_yticklabels(['{:.0f}%'.format(y) for y in y_ticks_1])

axs[0].set_xticks(range(len(df_first_plot.columns)))
axs[0].set_xticklabels(['CA', '1T BRT', '49 BRT', 'T'])

# Changing x labels for the second plot
axs[1].set_xticks(range(len(df_second_plot.columns)))
axs[1].set_xticklabels(['BA - Blue', 'BA - Green', 'BA - Orange', 'BA - Red', 'BA - Yellow'])

axs[0].set_xticks(range(len(df_first_plot.columns)))
axs[0].set_xticklabels(['Caltrain', '1Tempo BRT', 'Van Ness BRT', 'Central Subway'])

# Changing x labels for the second plot
axs[1].set_xticks(range(len(df_second_plot.columns)))
axs[1].set_xticklabels(['BA - Blue', 'BA - Green', 'BA - Orange', 'BA - Red', 'BA - Yellow'])


for ax in axs:
    ax.tick_params(axis='both', which='major', labelsize=18)

plt.tight_layout()
# plt.show()

plt.savefig('stacked_bar_chart_high_quality.png', dpi=600)


# **Transfers**

In [ ]:
import ast


dict_project = {
    'AC - 1TEMPO':'1T',
    'BA - Extension and Core project':'BA',
    'CA - Electrification Project':'CA',
    'SF - Central Subway':'T',
    'SF - Van Ness': '49'
}

veh_to_route_bs = pd.read_csv('outputs/routetovehicledictBaseline2018.csv')
veh_to_route_bs = veh_to_route_bs.set_index('trip_id')['route_id'].to_dict()
veh_to_route_TR = pd.read_csv('outputs/routetovehicledictFuture2018.csv')
veh_to_route_TR = veh_to_route_TR.set_index('trip_id')['route_id'].to_dict()

from_indexes = [

                '1', '49', 'K/T', 'J', 'L', 'M', 'N', 
                'BA - Blue', 'BA - Green', 'BA - Orange', 'BA - Red', 
                'BA - Yellow', 'Car', 'CarPool','TNC', 'TNC-Pool','Walk', 'Bike', 
    'CA', 'AC - others', 'SF - others','AM','AY','CC','CE','CM',
    'DE','EM','FF','GG','HF','MA','PE','RV', 'SB',
    'SC','SL','SM','SO','SR','ST','TD','UC','VC','VN','VT','WC','WH',
                'Other']
to_indexes_dict = {'CA':['CA:12867','CA:12868','CA:12869'], '1T':'AC:1-142', '49':'SF:12327', 'T':'SF:12476', 
              #'J', 'L', 'M', 'N', 
              'BA Blue':'BA:11', 
              'BA Green':'BA:5', 'BA Orange':'BA:3', 'BA Red':'BA:7', 'BA Yellow':'BA:1'}
to_indexes = ['CA', '1T', '49', 'T', 
              #'J', 'L', 'M', 'N', 
              'BA Blue', 
              'BA Green', 'BA Orange', 'BA Red', 'BA Yellow']
FT = pd.DataFrame(0, index=from_indexes, columns=to_indexes)

for veh_bs, veh_tr, project, mode_base in zip(list(df['Vehicles Used Baseline']), list(df['Vehicles Used TR']), list(df['Project Tried']), list(df['Chosen Mode Baseline'])):
    # routes_bs = veh_to_route_bs[veh_bs]
    # routes_TR = veh_to_route_TR[veh_tr]
    # veh_bs = ','.join(veh_bs)
    # veh_tr = ','.join(veh_tr)
    

    found = 0
    
    fixed_string = veh_bs.replace(" ", ", ")
    veh_bs = ast.literal_eval(fixed_string)
    fixed_string = veh_tr.replace(" ", ", ")
    veh_tr = ast.literal_eval(fixed_string)

    routes_bs = [veh_to_route_bs.get(key, '0') for key in veh_bs]
    routes_TR = [veh_to_route_TR.get(key, '0') for key in veh_tr]
    
    attr = len(routes_TR) - len(routes_bs)
    
    BARTS = []
    if project != 'BA - Extension and Core project' :
        BARTS.append(dict_project[project])
    else:
        
        if 'BA:1' in routes_TR or 'BA:2' in routes_TR :
            BARTS.append('BA Yellow')
        if 'BA:3' in routes_TR or 'BA:4' in routes_TR :
            BARTS.append('BA Orange')
        if 'BA:5' in routes_TR or 'BA:6' in routes_TR :
            BARTS.append('BA Green')
        if 'BA:7' in routes_TR or 'BA:8' in routes_TR :
            BARTS.append('BA Red')
        if 'BA:11' in routes_TR or 'BA:12' in routes_TR :
            BARTS.append('BA Blue')
            
    for BAR in BARTS:
        
        if BAR == '1T':
            if 'AC:1-142' in routes_bs:
                FT.loc['1',BAR] += attr
                found+=1
            if any('AC:' in s for s in routes_bs) and not any('AC:' in s for s in [item for item in routes_TR if item != 'AC:1T-142']) and not any('AC:1-142' in s for s in routes_bs) :
                FT.loc['AC - others',BAR] += attr
                found+=1
        if BAR == 'CA':
            if any('CA:' in s for s in routes_bs) :
                FT.loc['CA',BAR] += attr
                found+=1
        if BAR == 'BA Yellow':
            if 'BA:1' in routes_bs:
                FT.loc['BA - Yellow',BAR] += attr
                found+=1
        if BAR == 'BA Orange':
            if 'BA:3' in routes_bs:
                FT.loc['BA - Orange',BAR] += attr
                found+=1
        if BAR == 'BA Green':
            if 'BA:5' in routes_bs:
                FT.loc['BA - Green',BAR] += attr
                found+=1
        if BAR == 'BA Red':
            if 'BA:7' in routes_bs:
                FT.loc['BA - Red',BAR] += attr
                found+=1
        if BAR == 'BA Blue':
            if 'BA:11' in routes_bs:
                FT.loc['BA - Blue',BAR] += attr
                found+=1
        if BAR == 'T':
            if 'SF:12476' in routes_bs:
                FT.loc['K/T',BAR] += attr
                found+=1
            if any('SF:' in s for s in routes_bs) and not any('SF:' in s for s in [item for item in routes_TR if item != 'SF:1001']) and not any('SF:12327' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12476' in s for s in routes_bs) and not any('SF:12477' in s for s in routes_bs) and not any('SF:12478' in s for s in routes_bs) and not any('SF:12479' in s for s in routes_bs):
                FT.loc['SF - others',BAR] += attr
                found+=1
        if BAR == '49':
            if 'SF:12327' in routes_bs:
                FT.loc['49',BAR] += attr
                found+=1
            if any('SF:' in s for s in routes_bs) and not any('SF:' in s for s in [item for item in routes_TR if item != 'SF:18608']) and not any('SF:12327' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12476' in s for s in routes_bs) and not any('SF:12477' in s for s in routes_bs) and not any('SF:12478' in s for s in routes_bs) and not any('SF:12479' in s for s in routes_bs):
                FT.loc['SF - others',BAR] += attr
                found+=1
        

        # if any('AC:' in s for s in routes_bs):
        #     FT.loc['AC',BAR] += 1
        # if any('SF:' in s for s in routes_bs):
        #     FT.loc['SF',BAR] += 1
        # if any('BA:' in s for s in routes_bs):
        #     FT.loc['BA',BAR] += 1
            
        if 'AC:1-142' in routes_bs and 'AC:1T-142' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['1',BAR] += attr
            found+=1
        if 'SF:12327' in routes_bs and 'SF:18608' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['49',BAR] += attr
            found+=1
        if 'SF:12476' in routes_bs and 'SF:1001' not in routes_TR and 'SF:1002' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['K/T',BAR] += attr
            found+=1
        if 'SF:12475' in routes_bs and 'SF:12375' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['J',BAR] += attr
            found+=1
            # if BAR == 'CA':
            #     print(routes_bs, routes_TR, attr)
        if 'SF:12477' in routes_bs and 'SF:12377' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['L',BAR] += attr
            found+=1
        if 'SF:12478' in routes_bs and 'SF:12378' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['M',BAR] += attr
            found+=1
        if 'SF:12479' in routes_bs and 'SF:12379' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['N',BAR] += attr
            found+=1
        if 'BA:1' in routes_bs and 'BA:1' not in routes_TR and 'BA:2' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['BA - Yellow',BAR] += attr
            found+=1
        if 'BA:3' in routes_bs and 'BA:3' not in routes_TR and 'BA:4' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['BA - Orange',BAR] += attr
            found+=1
        if 'BA:5' in routes_bs and 'BA:5' not in routes_TR and 'BA:6' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['BA - Green',BAR] += attr
            found+=1
        if 'BA:7' in routes_bs and 'BA:7' not in routes_TR and 'BA:8' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['BA - Red',BAR] += attr
            found+=1
        if 'BA:11' in routes_bs and 'BA:11' not in routes_TR and 'BA:12' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['BA - Blue',BAR] += attr
            found+=1
            
        if mode_base =='walk' :
            FT.loc['Walk',BAR] += attr
            found+=1
        if mode_base =='ride_hail' :
            FT.loc['TNC',BAR] += attr
            found+=1
        if mode_base =='ride_hail_pooled' :
            FT.loc['TNC-Pool',BAR] += attr
            found+=1
        if mode_base =='car' :
            FT.loc['Car',BAR] += attr
            found+=1
        if mode_base =='car_hov2' :
            FT.loc['CarPool',BAR] += attr
            found+=1
        if mode_base =='car_hov3' :
            FT.loc['CarPool',BAR] += attr
            found+=1
        if mode_base =='bike' :
            FT.loc['Bike',BAR] += attr
            found+=1

            
        if any('AC:' in s for s in routes_bs) and not any('AC:' in s for s in routes_TR) and not any('AC:1-142' in s for s in routes_bs)  and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['AC - others',BAR] += attr
            found+=1
        if any('SF:' in s for s in routes_bs) and not any('SF:' in s for s in routes_TR) and not any('SF:12327' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12476' in s for s in routes_bs) and not any('SF:12477' in s for s in routes_bs) and not any('SF:12478' in s for s in routes_bs) and not any('SF:12479' in s for s in routes_bs) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SF - others',BAR] += attr
            found+=1
        # if any('BA:' in s for s in routes_bs) and not any('BA:' in s for s in routes_TR):
        #     FT.loc['BA',BAR] += 1
        #     found+=1
        
        if any('CA:' in s for s in routes_bs) and not any('CA:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['CA',BAR] += attr
            found+=1
        if any('AM:' in s for s in routes_bs) and not any('AM:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['AM',BAR] += attr
            found+=1
        if any('AY:' in s for s in routes_bs) and not any('AY:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['AY',BAR] += attr
            found+=1
        if any('CC:' in s for s in routes_bs) and not any('CC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['CC',BAR] += attr
            found+=1
        if any('CE:' in s for s in routes_bs) and not any('CE:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['CE',BAR] += attr
            found+=1
        if any('CM:' in s for s in routes_bs) and not any('CM:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['CM',BAR] += attr
            found+=1
        if any('DE:' in s for s in routes_bs) and not any('DE:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['DE',BAR] += attr
            found+=1
        if any('EM:' in s for s in routes_bs) and not any('EM:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['EM',BAR] += attr
            found+=1
        if any('GG:' in s for s in routes_bs) and not any('GG:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['GG',BAR] += attr
            found+=1
        if any('HF:' in s for s in routes_bs) and not any('HF:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['HF',BAR] += attr
            found+=1
        if any('MA:' in s for s in routes_bs) and not any('MA:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['MA',BAR] += attr
            found+=1
        if any('PE:' in s for s in routes_bs) and not any('PE:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['PE',BAR] += attr
            found+=1
        if any('RV:' in s for s in routes_bs) and not any('RV:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['RV',BAR] += attr
            found+=1
        if any('SB:' in s for s in routes_bs) and not any('SB:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SB',BAR] += attr
            found+=1
        if any('SC:' in s for s in routes_bs) and not any('SC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SC',BAR] += attr
            found+=1
        if any('SL:' in s for s in routes_bs) and not any('SL:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SL',BAR] += attr
            found+=1
        if any('SM:' in s for s in routes_bs) and not any('SM:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SM',BAR] += attr
            found+=1
        if any('SO:' in s for s in routes_bs) and not any('SO:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SO',BAR] += attr
            found+=1
        if any('SR:' in s for s in routes_bs) and not any('SR:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SR',BAR] += attr
            found+=1
        if any('ST:' in s for s in routes_bs) and not any('ST:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['ST',BAR] += attr
            found+=1
        if any('TD:' in s for s in routes_bs) and not any('TD:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['TD',BAR] += attr
            found+=1
        if any('UC:' in s for s in routes_bs) and not any('UC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['UC',BAR] += attr
            found+=1
        if any('VC:' in s for s in routes_bs) and not any('VC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['VC',BAR] += attr
            found+=1
        if any('VN:' in s for s in routes_bs) and not any('VN:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['VN',BAR] += attr
            found+=1
        if any('VT:' in s for s in routes_bs) and not any('VT:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['VT',BAR] += attr
            found+=1
        if any('WC:' in s for s in routes_bs) and not any('WC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['WC',BAR] += attr
            found+=1
        if any('WH:' in s for s in routes_bs) and not any('WH:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['WH',BAR] += attr
            found+=1    
            
            
            
            
        if found == 0:
            FT.loc['Other',BAR] += attr  
FT.loc['Total'] = [FT['CA'].sum(),FT['1T'].sum(),FT['49'].sum(),FT['T'].sum(),FT['BA Blue'].sum(),FT['BA Green'].sum(),FT['BA Orange'].sum(),FT['BA Red'].sum(),FT['BA Yellow'].sum()]

FT = (FT/Scale).astype(int)

In [ ]:
FT

In [ ]:
FT/FT3

# **Durations**

In [ ]:
import ast



dict_project = {
    'AC - 1TEMPO':'1T',
    'BA - Extension and Core project':'BA',
    'CA - Electrification Project':'CA',
    'SF - Central Subway':'T',
    'SF - Van Ness': '49'
}

veh_to_route_bs = pd.read_csv('outputs/routetovehicledictBaseline2018.csv')
veh_to_route_bs = veh_to_route_bs.set_index('trip_id')['route_id'].to_dict()
veh_to_route_TR = pd.read_csv('outputs/routetovehicledictFuture2018.csv')
veh_to_route_TR = veh_to_route_TR.set_index('trip_id')['route_id'].to_dict()

from_indexes = [

                '1', '49', 'K/T', 'J', 'L', 'M', 'N', 
                'BA - Blue', 'BA - Green', 'BA - Orange', 'BA - Red', 
                'BA - Yellow', 'Car', 'CarPool','TNC', 'TNC-Pool','Walk', 'Bike', 
    'CA', 'AC - others', 'SF - others','AM','AY','CC','CE','CM',
    'DE','EM','FF','GG','HF','MA','PE','RV', 'SB',
    'SC','SL','SM','SO','SR','ST','TD','UC','VC','VN','VT','WC','WH',
                'Other']
to_indexes_dict = {'CA':['CA:12867','CA:12868','CA:12869'], '1T':'AC:1-142', '49':'SF:12327', 'T':'SF:12476', 
              #'J', 'L', 'M', 'N', 
              'BA Blue':'BA:11', 
              'BA Green':'BA:5', 'BA Orange':'BA:3', 'BA Red':'BA:7', 'BA Yellow':'BA:1'}
to_indexes = ['CA', '1T', '49', 'T', 
              #'J', 'L', 'M', 'N', 
              'BA Blue', 
              'BA Green', 'BA Orange', 'BA Red', 'BA Yellow']
FT = pd.DataFrame(0, index=from_indexes, columns=to_indexes)


for veh_bs, veh_tr, project, mode_base, dur_bs, dur_tr in zip(list(df['Vehicles Used Baseline']), list(df['Vehicles Used TR']), list(df['Project Tried'])
                                              , list(df['Chosen Mode Baseline']),   list(df['Trip Duration Baseline']),   list(df['Trip Duration TR'])):
    # routes_bs = veh_to_route_bs[veh_bs]
    # routes_TR = veh_to_route_TR[veh_tr]
    # veh_bs = ','.join(veh_bs)
    # veh_tr = ','.join(veh_tr)
    

    found = 0
    
    fixed_string = veh_bs.replace(" ", ", ")
    veh_bs = ast.literal_eval(fixed_string)
    fixed_string = veh_tr.replace(" ", ", ")
    veh_tr = ast.literal_eval(fixed_string)

    routes_bs = [veh_to_route_bs.get(key, '0') for key in veh_bs]
    routes_TR = [veh_to_route_TR.get(key, '0') for key in veh_tr]
    
    attr = dur_tr - dur_bs
    
    BARTS = []
    if project != 'BA - Extension and Core project' :
        BARTS.append(dict_project[project])
    else:
        
        if 'BA:1' in routes_TR or 'BA:2' in routes_TR :
            BARTS.append('BA Yellow')
        if 'BA:3' in routes_TR or 'BA:4' in routes_TR :
            BARTS.append('BA Orange')
        if 'BA:5' in routes_TR or 'BA:6' in routes_TR :
            BARTS.append('BA Green')
        if 'BA:7' in routes_TR or 'BA:8' in routes_TR :
            BARTS.append('BA Red')
        if 'BA:11' in routes_TR or 'BA:12' in routes_TR :
            BARTS.append('BA Blue')
            
    for BAR in BARTS:
        
        if BAR == '1T':
            if 'AC:1-142' in routes_bs:
                FT.loc['1',BAR] += attr
                found+=1
            if any('AC:' in s for s in routes_bs) and not any('AC:' in s for s in [item for item in routes_TR if item != 'AC:1T-142']) and not any('AC:1-142' in s for s in routes_bs) :
                FT.loc['AC - others',BAR] += attr
                found+=1
        if BAR == 'CA':
            if any('CA:' in s for s in routes_bs) :
                FT.loc['CA',BAR] += attr
                found+=1
        if BAR == 'BA Yellow':
            if 'BA:1' in routes_bs:
                FT.loc['BA - Yellow',BAR] += attr
                found+=1
        if BAR == 'BA Orange':
            if 'BA:3' in routes_bs:
                FT.loc['BA - Orange',BAR] += attr
                found+=1
        if BAR == 'BA Green':
            if 'BA:5' in routes_bs:
                FT.loc['BA - Green',BAR] += attr
                found+=1
        if BAR == 'BA Red':
            if 'BA:7' in routes_bs:
                FT.loc['BA - Red',BAR] += attr
                found+=1
        if BAR == 'BA Blue':
            if 'BA:11' in routes_bs:
                FT.loc['BA - Blue',BAR] += attr
                found+=1
        if BAR == 'T':
            if 'SF:12476' in routes_bs:
                FT.loc['K/T',BAR] += attr
                found+=1
            if any('SF:' in s for s in routes_bs) and not any('SF:' in s for s in [item for item in routes_TR if item != 'SF:1001']) and not any('SF:12327' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12476' in s for s in routes_bs) and not any('SF:12477' in s for s in routes_bs) and not any('SF:12478' in s for s in routes_bs) and not any('SF:12479' in s for s in routes_bs):
                FT.loc['SF - others',BAR] += attr
                found+=1
        if BAR == '49':
            if 'SF:12327' in routes_bs:
                FT.loc['49',BAR] += attr
                found+=1
            if any('SF:' in s for s in routes_bs) and not any('SF:' in s for s in [item for item in routes_TR if item != 'SF:18608']) and not any('SF:12327' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12476' in s for s in routes_bs) and not any('SF:12477' in s for s in routes_bs) and not any('SF:12478' in s for s in routes_bs) and not any('SF:12479' in s for s in routes_bs):
                FT.loc['SF - others',BAR] += attr
                found+=1
        

        # if any('AC:' in s for s in routes_bs):
        #     FT.loc['AC',BAR] += 1
        # if any('SF:' in s for s in routes_bs):
        #     FT.loc['SF',BAR] += 1
        # if any('BA:' in s for s in routes_bs):
        #     FT.loc['BA',BAR] += 1
            
        if 'AC:1-142' in routes_bs and 'AC:1T-142' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['1',BAR] += attr
            found+=1
        if 'SF:12327' in routes_bs and 'SF:18608' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['49',BAR] += attr
            found+=1
        if 'SF:12476' in routes_bs and 'SF:1001' not in routes_TR and 'SF:1002' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['K/T',BAR] += attr
            found+=1
        if 'SF:12475' in routes_bs and 'SF:12375' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['J',BAR] += attr
            found+=1
            # if BAR == 'CA':
            #     print(routes_bs, routes_TR, attr)
        if 'SF:12477' in routes_bs and 'SF:12377' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['L',BAR] += attr
            found+=1
        if 'SF:12478' in routes_bs and 'SF:12378' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['M',BAR] += attr
            found+=1
        if 'SF:12479' in routes_bs and 'SF:12379' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['N',BAR] += attr
            found+=1
        if 'BA:1' in routes_bs and 'BA:1' not in routes_TR and 'BA:2' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['BA - Yellow',BAR] += attr
            found+=1
        if 'BA:3' in routes_bs and 'BA:3' not in routes_TR and 'BA:4' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['BA - Orange',BAR] += attr
            found+=1
        if 'BA:5' in routes_bs and 'BA:5' not in routes_TR and 'BA:6' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['BA - Green',BAR] += attr
            found+=1
        if 'BA:7' in routes_bs and 'BA:7' not in routes_TR and 'BA:8' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['BA - Red',BAR] += attr
            found+=1
        if 'BA:11' in routes_bs and 'BA:11' not in routes_TR and 'BA:12' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['BA - Blue',BAR] += attr
            found+=1
            
        if mode_base =='walk' :
            FT.loc['Walk',BAR] += attr
            found+=1
        if mode_base =='ride_hail' :
            FT.loc['TNC',BAR] += attr
            found+=1
        if mode_base =='ride_hail_pooled' :
            FT.loc['TNC-Pool',BAR] += attr
            found+=1
        if mode_base =='car' :
            FT.loc['Car',BAR] += attr
            found+=1
        if mode_base =='car_hov2' :
            FT.loc['CarPool',BAR] += attr
            found+=1
        if mode_base =='car_hov3' :
            FT.loc['CarPool',BAR] += attr
            found+=1
        if mode_base =='bike' :
            FT.loc['Bike',BAR] += attr
            found+=1

            
        if any('AC:' in s for s in routes_bs) and not any('AC:' in s for s in routes_TR) and not any('AC:1-142' in s for s in routes_bs)  and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['AC - others',BAR] += attr
            found+=1
        if any('SF:' in s for s in routes_bs) and not any('SF:' in s for s in routes_TR) and not any('SF:12327' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12476' in s for s in routes_bs) and not any('SF:12477' in s for s in routes_bs) and not any('SF:12478' in s for s in routes_bs) and not any('SF:12479' in s for s in routes_bs) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SF - others',BAR] += attr
            found+=1
        # if any('BA:' in s for s in routes_bs) and not any('BA:' in s for s in routes_TR):
        #     FT.loc['BA',BAR] += 1
        #     found+=1
        
        if any('CA:' in s for s in routes_bs) and not any('CA:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['CA',BAR] += attr
            found+=1
        if any('AM:' in s for s in routes_bs) and not any('AM:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['AM',BAR] += attr
            found+=1
        if any('AY:' in s for s in routes_bs) and not any('AY:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['AY',BAR] += attr
            found+=1
        if any('CC:' in s for s in routes_bs) and not any('CC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['CC',BAR] += attr
            found+=1
        if any('CE:' in s for s in routes_bs) and not any('CE:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['CE',BAR] += attr
            found+=1
        if any('CM:' in s for s in routes_bs) and not any('CM:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['CM',BAR] += attr
            found+=1
        if any('DE:' in s for s in routes_bs) and not any('DE:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['DE',BAR] += attr
            found+=1
        if any('EM:' in s for s in routes_bs) and not any('EM:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['EM',BAR] += attr
            found+=1
        if any('GG:' in s for s in routes_bs) and not any('GG:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['GG',BAR] += attr
            found+=1
        if any('HF:' in s for s in routes_bs) and not any('HF:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['HF',BAR] += attr
            found+=1
        if any('MA:' in s for s in routes_bs) and not any('MA:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['MA',BAR] += attr
            found+=1
        if any('PE:' in s for s in routes_bs) and not any('PE:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['PE',BAR] += attr
            found+=1
        if any('RV:' in s for s in routes_bs) and not any('RV:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['RV',BAR] += attr
            found+=1
        if any('SB:' in s for s in routes_bs) and not any('SB:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SB',BAR] += attr
            found+=1
        if any('SC:' in s for s in routes_bs) and not any('SC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SC',BAR] += attr
            found+=1
        if any('SL:' in s for s in routes_bs) and not any('SL:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SL',BAR] += attr
            found+=1
        if any('SM:' in s for s in routes_bs) and not any('SM:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SM',BAR] += attr
            found+=1
        if any('SO:' in s for s in routes_bs) and not any('SO:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SO',BAR] += attr
            found+=1
        if any('SR:' in s for s in routes_bs) and not any('SR:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SR',BAR] += attr
            found+=1
        if any('ST:' in s for s in routes_bs) and not any('ST:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['ST',BAR] += attr
            found+=1
        if any('TD:' in s for s in routes_bs) and not any('TD:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['TD',BAR] += attr
            found+=1
        if any('UC:' in s for s in routes_bs) and not any('UC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['UC',BAR] += attr
            found+=1
        if any('VC:' in s for s in routes_bs) and not any('VC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['VC',BAR] += attr
            found+=1
        if any('VN:' in s for s in routes_bs) and not any('VN:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['VN',BAR] += attr
            found+=1
        if any('VT:' in s for s in routes_bs) and not any('VT:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['VT',BAR] += attr
            found+=1
        if any('WC:' in s for s in routes_bs) and not any('WC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['WC',BAR] += attr
            found+=1
        if any('WH:' in s for s in routes_bs) and not any('WH:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['WH',BAR] += attr
            found+=1    
            
            
            
            
        if found == 0:
            FT.loc['Other',BAR] += attr  
FT.loc['Total'] = [FT['CA'].sum(),FT['1T'].sum(),FT['49'].sum(),FT['T'].sum(),FT['BA Blue'].sum(),FT['BA Green'].sum(),FT['BA Orange'].sum(),FT['BA Red'].sum(),FT['BA Yellow'].sum()]

FT = (FT/Scale).astype(int)

In [ ]:
FT

In [ ]:
FT/FT3

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def add_row_name_to_bars(ax, bottoms, values, column_names, row_name, threshold):
    new_bottoms = [bottom + value for bottom, value in zip(bottoms, values)]
    for i, (bottom, new_bottom, value) in enumerate(zip(bottoms, new_bottoms, values)):
        mid_point = (bottom + new_bottom) / 2
        if abs(value) > threshold * 350:  # Use different thresholds for each plot
            ax.text(i, mid_point, row_name, ha='center', va='center', color='black', fontsize=12)
    return new_bottoms
def separate_pos_neg(values):
    pos_values = [v if v >= 0 else 0 for v in values]
    neg_values = [v if v < 0 else 0 for v in values]
    pos_bottoms = [0] * len(values)
    neg_bottoms = [0] * len(values)
    for i, (pos_val, neg_val) in enumerate(zip(pos_values, neg_values)):
        if pos_val > 0:
            pos_bottoms[i] = pos_val
        if neg_val < 0:
            neg_bottoms[i] = neg_val
    return pos_values, neg_values, pos_bottoms, neg_bottoms

# Sample data preparation
# [Your data preparation code here, e.g., loading the df3 DataFrame]
df3 = FT/3600
df3 = df3.drop("Total", axis=0)
# df3 = df3.drop("Other", axis=0)
# Color mapping for the first plot
color_mapping1= {
    '1': '#A8D5BA',        # Light Green
    'AC - others': '#A8D5BA',
    '49': '#F2B5B5',      # Light Red
    'K/T': '#F2B5B5',
    'J': '#F2B5B5',
    'L': '#F2B5B5',
    'M': '#F2B5B5',
    'N': '#F2B5B5',
    'SF - others': '#F2B5B5',
    'BART': '#A9AEEF',    # Light Blue
    'BA - Blue': '#A9AEEF',
    'BA - Green': '#BEE9C4',  # Pastel Green
    'BA - Orange': '#FFDAC1',  # Pastel Orange
    'BA - Red': '#FFB6B6',     # Pastel Red
    'BA - Yellow': '#FFF5B7',  # Pastel Yellow
    'Car': '#D7CCC8',         # Soft Red
    'CarPool': '#D7CCC8',     # Light Grey
    'TNC': '#D7CCC8',
    'TNC-Pool': '#D7CCC8',
    'Walk': '#D7CCC8',
    'Bike': '#D7CCC8',
    'CA': '#80CBC4',          # Soft Blue
    'AM': '#CEA2E1',         # Soft Purple
    'AY': '#CEA2E1',
    'CC': '#CEA2E1',
    'CE': '#CEA2E1',
    'CM': '#CEA2E1',
    'DE': '#CEA2E1',
    'EM': '#CEA2E1',
    'FF': '#CEA2E1',
    'GG': '#CEA2E1',
    'HF': '#CEA2E1',
    'MA': '#CEA2E1',
    'PE': '#CEA2E1',
    'RV': '#CEA2E1',
    'SB': '#CEA2E1',
    'SC': '#CEA2E1',
    'SL': '#CEA2E1',
    'SM': '#CEA2E1',
    'SO': '#CEA2E1',
    'SR': '#CEA2E1',
    'ST': '#CEA2E1',
    'TD': '#CEA2E1',
    'UC': '#CEA2E1',
    'VC': '#CEA2E1',
    'VN': '#CEA2E1',
    'VT': '#CEA2E1',
    'WC': '#CEA2E1',
    'WH': '#CEA2E1',
    'Other': '#F5F5F5'        # Light Grey
}
color_mapping2 = {
    '1': '#F5F5F5',  # muted dark green
    'AC - others': '#F5F5F5',
    '49': '#F5F5F5',  # muted dark red
    'K/T': '#F5F5F5',
    'J': '#F5F5F5',
    'L': '#F5F5F5',
    'M': '#F5F5F5',
    'N': '#F5F5F5',
    'SF - others': '#F5F5F5',
    'BART': '#F5F5F5',
    'BA - Blue': '#BBDEFB',  # muted blue
    'BA - Green': '#C8E6C9',  # muted green
    'BA - Orange': '#FFE0B2',  # muted orange
    'BA - Red': '#FFCDD2',  # muted red
    'BA - Yellow': '#FFF9C4',  # muted yellow
    'Car': '#F5F5F5',  # dark red
    'CarPool': '#F5F5F5',  # grey
    'TNC': '#F5F5F5',
    'TNC-Pool': '#F5F5F5',
    'Walk': '#F5F5F5',
    'Bike': '#F5F5F5',
    'CA': '#F5F5F5',  # muted dark blue
    'AM': '#F5F5F5',  # muted dark purple
    'AY': '#F5F5F5',
    'CC': '#F5F5F5',
    'CE': '#F5F5F5',
    'CM': '#F5F5F5',
    'DE': '#F5F5F5',
    'EM': '#F5F5F5',
    'FF': '#F5F5F5',
    'GG': '#F5F5F5',
    'HF': '#F5F5F5',
    'MA': '#F5F5F5',
    'PE': '#F5F5F5',
    'RV': '#F5F5F5',
    'SB': '#F5F5F5',
    'SC': '#F5F5F5',
    'SL': '#F5F5F5',
    'SM': '#F5F5F5',
    'SO': '#F5F5F5',
    'SR': '#F5F5F5',
    'ST': '#F5F5F5',
    'TD': '#F5F5F5',
    'UC': '#F5F5F5',
    'VC': '#F5F5F5',
    'VN': '#F5F5F5',
    'VT': '#F5F5F5',
    'WC': '#F5F5F5',
    'WH': '#F5F5F5',
    'Other': '#F5F5F5'  # grey
}
desired_order2 = [
    'AC - others', '1', 
    'SF - others', '49', 
    'K/T', 'J', 'L', 'M', 'N', 
    'BA - Blue', 'BA - Green', 'BA - Orange', 'BA - Red', 'BA - Yellow', 
    'Car', 'CarPool', 'TNC', 'TNC-Pool', 'Walk', 'Bike', 
    'CA',
    'AM', 'AY', 'CC', 'CE', 'CM', 'DE', 'EM', 'FF', 'GG', 'HF', 'MA', 'PE', 'RV', 'SB', 'SC', 
    'SL', 'SM', 'SO', 'SR', 'ST', 'TD', 'UC', 'VC', 'VN', 'VT', 'WC', 'WH',
    'Other'
]
desired_order1 = [
    'AC - others', '1', 
    'SF - others', '49', 
    'K/T', 'J', 'L', 'M', 'N', 
    'BART', 
    'Car', 'CarPool', 'TNC', 'TNC-Pool', 'Walk', 'Bike', 
    'CA',
    'AM', 'AY', 'CC', 'CE', 'CM', 'DE', 'EM', 'FF', 'GG', 'HF', 'MA', 'PE', 'RV', 'SB', 'SC', 
    'SL', 'SM', 'SO', 'SR', 'ST', 'TD', 'UC', 'VC', 'VN', 'VT', 'WC', 'WH',
    'Other'
]
# Assuming you have df3 and color_mapping set as before

# Aggregate BA - type rows into BART
ba_rows = ['BA - Blue', 'BA - Green', 'BA - Orange', 'BA - Red', 'BA - Yellow']
df3.loc['BART'] = df3.loc[ba_rows].sum()

# Filter and reorder the rows for both plots
df_first_plot = df3.loc[desired_order1,['CA', '1T','49', 'T']]
df_second_plot = df3.loc[desired_order2,['BA Blue', 'BA Green', 'BA Orange', 'BA Red', 'BA Yellow']]

# Create subplots with 1 row and 2 columns
# Function to separate positive and negative values for stacking


# Create subplots with 1 row and 2 columns
fig, axs = plt.subplots(1, 2, figsize=(16, 8))

# Plotting the first graph
pos_bottoms = [0] * len(df_first_plot.columns)
neg_bottoms = [0] * len(df_first_plot.columns)
for row_name in df_first_plot.index:
    values = df_first_plot.loc[row_name]
    pos_values = [v if v >= 0 else 0 for v in values]
    neg_values = [v if v < 0 else 0 for v in values]

    axs[0].bar(df_first_plot.columns, pos_values, bottom=pos_bottoms, color=color_mapping1[row_name], edgecolor='white')
    axs[0].bar(df_first_plot.columns, neg_values, bottom=neg_bottoms, color=color_mapping1[row_name], edgecolor='white')

    pos_bottoms = add_row_name_to_bars(axs[0], pos_bottoms, pos_values, df_first_plot.columns, row_name, threshold=650/3600)
    neg_bottoms = add_row_name_to_bars(axs[0], neg_bottoms, neg_values, df_first_plot.columns, row_name, threshold=650/3600)

# Plotting the second graph
pos_bottoms = [0] * len(df_second_plot.columns)
neg_bottoms = [0] * len(df_second_plot.columns)
for row_name in df_second_plot.index:
    values = df_second_plot.loc[row_name]
    pos_values = [v if v >= 0 else 0 for v in values]
    neg_values = [v if v < 0 else 0 for v in values]

    axs[1].bar(df_second_plot.columns, pos_values, bottom=pos_bottoms, color=color_mapping2[row_name], edgecolor='white')
    axs[1].bar(df_second_plot.columns, neg_values, bottom=neg_bottoms, color=color_mapping2[row_name], edgecolor='white')

    pos_bottoms = add_row_name_to_bars(axs[1], pos_bottoms, pos_values, df_second_plot.columns, row_name, threshold=1800/3600)
    neg_bottoms = add_row_name_to_bars(axs[1], neg_bottoms, neg_values, df_second_plot.columns, row_name, threshold=1800/3600)
# axs[0].set_ylim(-90*11.23, 115*11.23)# Adjust the upper limit as necessary
# axs[1].set_ylim(-150*11.23,1350*11.23) # Adjust the upper limit as necessary
axs[0].axhline(0, color='black', linewidth=0.5)
axs[1].axhline(0, color='black', linewidth=0.5)


axs[0].set_xticks(range(len(df_first_plot.columns)))
axs[0].set_xticklabels(['Caltrain', '1Tempo BRT', 'Van Ness BRT', 'Central Subway'])

# Changing x labels for the second plot
axs[1].set_xticks(range(len(df_second_plot.columns)))
axs[1].set_xticklabels(['BA - Blue', 'BA - Green', 'BA - Orange', 'BA - Red', 'BA - Yellow'])


for ax in axs:
    ax.tick_params(axis='both', which='major', labelsize=14)

    
plt.tight_layout()
plt.savefig('divergence_stacked_bar_chart_high_qualitydur.png', dpi=600)


# **Lengths**

In [ ]:
import ast


dict_project = {
    'AC - 1TEMPO':'1T',
    'BA - Extension and Core project':'BA',
    'CA - Electrification Project':'CA',
    'SF - Central Subway':'T',
    'SF - Van Ness': '49'
}

veh_to_route_bs = pd.read_csv('outputs/routetovehicledictBaseline2018.csv')
veh_to_route_bs = veh_to_route_bs.set_index('trip_id')['route_id'].to_dict()
veh_to_route_TR = pd.read_csv('outputs/routetovehicledictFuture2018.csv')
veh_to_route_TR = veh_to_route_TR.set_index('trip_id')['route_id'].to_dict()

from_indexes = [

                '1', '49', 'K/T', 'J', 'L', 'M', 'N', 
                'BA - Blue', 'BA - Green', 'BA - Orange', 'BA - Red', 
                'BA - Yellow', 'Car', 'CarPool','TNC', 'TNC-Pool','Walk', 'Bike', 
    'CA', 'AC - others', 'SF - others','AM','AY','CC','CE','CM',
    'DE','EM','FF','GG','HF','MA','PE','RV', 'SB',
    'SC','SL','SM','SO','SR','ST','TD','UC','VC','VN','VT','WC','WH',
                'Other']
to_indexes_dict = {'CA':['CA:12867','CA:12868','CA:12869'], '1T':'AC:1-142', '49':'SF:12327', 'T':'SF:12476', 
              #'J', 'L', 'M', 'N', 
              'BA Blue':'BA:11', 
              'BA Green':'BA:5', 'BA Orange':'BA:3', 'BA Red':'BA:7', 'BA Yellow':'BA:1'}
to_indexes = ['CA', '1T', '49', 'T', 
              #'J', 'L', 'M', 'N', 
              'BA Blue', 
              'BA Green', 'BA Orange', 'BA Red', 'BA Yellow']
FT = pd.DataFrame(0, index=from_indexes, columns=to_indexes)





for veh_bs, veh_tr, project, mode_base, dist_bs, dist_tr in zip(list(df['Vehicles Used Baseline']), list(df['Vehicles Used TR']), list(df['Project Tried']), 
                                              list(df['Chosen Mode Baseline']), list(df['Trip Length Baseline']), list(df['Trip Length TR'])):
    # routes_bs = veh_to_route_bs[veh_bs]
    # routes_TR = veh_to_route_TR[veh_tr]
    # veh_bs = ','.join(veh_bs)
    # veh_tr = ','.join(veh_tr)
    

    found = 0
    
    fixed_string = veh_bs.replace(" ", ", ")
    veh_bs = ast.literal_eval(fixed_string)
    fixed_string = veh_tr.replace(" ", ", ")
    veh_tr = ast.literal_eval(fixed_string)

    routes_bs = [veh_to_route_bs.get(key, '0') for key in veh_bs]
    routes_TR = [veh_to_route_TR.get(key, '0') for key in veh_tr]
    
    attr = dist_tr - dist_bs
    
    BARTS = []
    if project != 'BA - Extension and Core project' :
        BARTS.append(dict_project[project])
    else:
        
        if 'BA:1' in routes_TR or 'BA:2' in routes_TR :
            BARTS.append('BA Yellow')
        if 'BA:3' in routes_TR or 'BA:4' in routes_TR :
            BARTS.append('BA Orange')
        if 'BA:5' in routes_TR or 'BA:6' in routes_TR :
            BARTS.append('BA Green')
        if 'BA:7' in routes_TR or 'BA:8' in routes_TR :
            BARTS.append('BA Red')
        if 'BA:11' in routes_TR or 'BA:12' in routes_TR :
            BARTS.append('BA Blue')
            
    for BAR in BARTS:
        
        if BAR == '1T':
            if 'AC:1-142' in routes_bs:
                FT.loc['1',BAR] += attr
                found+=1
            if any('AC:' in s for s in routes_bs) and not any('AC:' in s for s in [item for item in routes_TR if item != 'AC:1T-142']) and not any('AC:1-142' in s for s in routes_bs) :
                FT.loc['AC - others',BAR] += attr
                found+=1
        if BAR == 'CA':
            if any('CA:' in s for s in routes_bs) :
                FT.loc['CA',BAR] += attr
                found+=1
        if BAR == 'BA Yellow':
            if 'BA:1' in routes_bs:
                FT.loc['BA - Yellow',BAR] += attr
                found+=1
        if BAR == 'BA Orange':
            if 'BA:3' in routes_bs:
                FT.loc['BA - Orange',BAR] += attr
                found+=1
        if BAR == 'BA Green':
            if 'BA:5' in routes_bs:
                FT.loc['BA - Green',BAR] += attr
                found+=1
        if BAR == 'BA Red':
            if 'BA:7' in routes_bs:
                FT.loc['BA - Red',BAR] += attr
                found+=1
        if BAR == 'BA Blue':
            if 'BA:11' in routes_bs:
                FT.loc['BA - Blue',BAR] += attr
                found+=1
        if BAR == 'T':
            if 'SF:12476' in routes_bs:
                FT.loc['K/T',BAR] += attr
                found+=1
            if any('SF:' in s for s in routes_bs) and not any('SF:' in s for s in [item for item in routes_TR if item != 'SF:1001']) and not any('SF:12327' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12476' in s for s in routes_bs) and not any('SF:12477' in s for s in routes_bs) and not any('SF:12478' in s for s in routes_bs) and not any('SF:12479' in s for s in routes_bs):
                FT.loc['SF - others',BAR] += attr
                found+=1
        if BAR == '49':
            if 'SF:12327' in routes_bs:
                FT.loc['49',BAR] += attr
                found+=1
            if any('SF:' in s for s in routes_bs) and not any('SF:' in s for s in [item for item in routes_TR if item != 'SF:18608']) and not any('SF:12327' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12476' in s for s in routes_bs) and not any('SF:12477' in s for s in routes_bs) and not any('SF:12478' in s for s in routes_bs) and not any('SF:12479' in s for s in routes_bs):
                FT.loc['SF - others',BAR] += attr
                found+=1
        

        # if any('AC:' in s for s in routes_bs):
        #     FT.loc['AC',BAR] += 1
        # if any('SF:' in s for s in routes_bs):
        #     FT.loc['SF',BAR] += 1
        # if any('BA:' in s for s in routes_bs):
        #     FT.loc['BA',BAR] += 1
            
        if 'AC:1-142' in routes_bs and 'AC:1T-142' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['1',BAR] += attr
            found+=1
        if 'SF:12327' in routes_bs and 'SF:18608' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['49',BAR] += attr
            found+=1
        if 'SF:12476' in routes_bs and 'SF:1001' not in routes_TR and 'SF:1002' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['K/T',BAR] += attr
            found+=1
        if 'SF:12475' in routes_bs and 'SF:12375' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['J',BAR] += attr
            found+=1
            # if BAR == 'CA':
            #     print(routes_bs, routes_TR, attr)
        if 'SF:12477' in routes_bs and 'SF:12377' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['L',BAR] += attr
            found+=1
        if 'SF:12478' in routes_bs and 'SF:12378' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['M',BAR] += attr
            found+=1
        if 'SF:12479' in routes_bs and 'SF:12379' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['N',BAR] += attr
            found+=1
        if 'BA:1' in routes_bs and 'BA:1' not in routes_TR and 'BA:2' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['BA - Yellow',BAR] += attr
            found+=1
        if 'BA:3' in routes_bs and 'BA:3' not in routes_TR and 'BA:4' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['BA - Orange',BAR] += attr
            found+=1
        if 'BA:5' in routes_bs and 'BA:5' not in routes_TR and 'BA:6' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['BA - Green',BAR] += attr
            found+=1
        if 'BA:7' in routes_bs and 'BA:7' not in routes_TR and 'BA:8' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['BA - Red',BAR] += attr
            found+=1
        if 'BA:11' in routes_bs and 'BA:11' not in routes_TR and 'BA:12' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['BA - Blue',BAR] += attr
            found+=1
            
        if mode_base =='walk' :
            FT.loc['Walk',BAR] += attr
            found+=1
        if mode_base =='ride_hail' :
            FT.loc['TNC',BAR] += attr
            found+=1
        if mode_base =='ride_hail_pooled' :
            FT.loc['TNC-Pool',BAR] += attr
            found+=1
        if mode_base =='car' :
            FT.loc['Car',BAR] += attr
            found+=1
        if mode_base =='car_hov2' :
            FT.loc['CarPool',BAR] += attr
            found+=1
        if mode_base =='car_hov3' :
            FT.loc['CarPool',BAR] += attr
            found+=1
        if mode_base =='bike' :
            FT.loc['Bike',BAR] += attr
            found+=1

            
        if any('AC:' in s for s in routes_bs) and not any('AC:' in s for s in routes_TR) and not any('AC:1-142' in s for s in routes_bs)  and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['AC - others',BAR] += attr
            found+=1
        if any('SF:' in s for s in routes_bs) and not any('SF:' in s for s in routes_TR) and not any('SF:12327' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12476' in s for s in routes_bs) and not any('SF:12477' in s for s in routes_bs) and not any('SF:12478' in s for s in routes_bs) and not any('SF:12479' in s for s in routes_bs) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SF - others',BAR] += attr
            found+=1
        # if any('BA:' in s for s in routes_bs) and not any('BA:' in s for s in routes_TR):
        #     FT.loc['BA',BAR] += 1
        #     found+=1
        
        if any('CA:' in s for s in routes_bs) and not any('CA:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['CA',BAR] += attr
            found+=1
        if any('AM:' in s for s in routes_bs) and not any('AM:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['AM',BAR] += attr
            found+=1
        if any('AY:' in s for s in routes_bs) and not any('AY:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['AY',BAR] += attr
            found+=1
        if any('CC:' in s for s in routes_bs) and not any('CC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['CC',BAR] += attr
            found+=1
        if any('CE:' in s for s in routes_bs) and not any('CE:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['CE',BAR] += attr
            found+=1
        if any('CM:' in s for s in routes_bs) and not any('CM:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['CM',BAR] += attr
            found+=1
        if any('DE:' in s for s in routes_bs) and not any('DE:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['DE',BAR] += attr
            found+=1
        if any('EM:' in s for s in routes_bs) and not any('EM:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['EM',BAR] += attr
            found+=1
        if any('GG:' in s for s in routes_bs) and not any('GG:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['GG',BAR] += attr
            found+=1
        if any('HF:' in s for s in routes_bs) and not any('HF:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['HF',BAR] += attr
            found+=1
        if any('MA:' in s for s in routes_bs) and not any('MA:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['MA',BAR] += attr
            found+=1
        if any('PE:' in s for s in routes_bs) and not any('PE:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['PE',BAR] += attr
            found+=1
        if any('RV:' in s for s in routes_bs) and not any('RV:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['RV',BAR] += attr
            found+=1
        if any('SB:' in s for s in routes_bs) and not any('SB:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SB',BAR] += attr
            found+=1
        if any('SC:' in s for s in routes_bs) and not any('SC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SC',BAR] += attr
            found+=1
        if any('SL:' in s for s in routes_bs) and not any('SL:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SL',BAR] += attr
            found+=1
        if any('SM:' in s for s in routes_bs) and not any('SM:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SM',BAR] += attr
            found+=1
        if any('SO:' in s for s in routes_bs) and not any('SO:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SO',BAR] += attr
            found+=1
        if any('SR:' in s for s in routes_bs) and not any('SR:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SR',BAR] += attr
            found+=1
        if any('ST:' in s for s in routes_bs) and not any('ST:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['ST',BAR] += attr
            found+=1
        if any('TD:' in s for s in routes_bs) and not any('TD:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['TD',BAR] += attr
            found+=1
        if any('UC:' in s for s in routes_bs) and not any('UC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['UC',BAR] += attr
            found+=1
        if any('VC:' in s for s in routes_bs) and not any('VC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['VC',BAR] += attr
            found+=1
        if any('VN:' in s for s in routes_bs) and not any('VN:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['VN',BAR] += attr
            found+=1
        if any('VT:' in s for s in routes_bs) and not any('VT:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['VT',BAR] += attr
            found+=1
        if any('WC:' in s for s in routes_bs) and not any('WC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['WC',BAR] += attr
            found+=1
        if any('WH:' in s for s in routes_bs) and not any('WH:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['WH',BAR] += attr
            found+=1    
            
            
            
            
        if found == 0:
            FT.loc['Other',BAR] += attr  
FT.loc['Total'] = [FT['CA'].sum(),FT['1T'].sum(),FT['49'].sum(),FT['T'].sum(),FT['BA Blue'].sum(),FT['BA Green'].sum(),FT['BA Orange'].sum(),FT['BA Red'].sum(),FT['BA Yellow'].sum()]

FT = (FT/Scale).astype(int)

# **inexus**

In [ ]:
import ast



dict_project = {
    'AC - 1TEMPO':'1T',
    'BA - Extension and Core project':'BA',
    'CA - Electrification Project':'CA',
    'SF - Central Subway':'T',
    'SF - Van Ness': '49'
}

veh_to_route_bs = pd.read_csv('outputs/routetovehicledictBaseline2018.csv')
veh_to_route_bs = veh_to_route_bs.set_index('trip_id')['route_id'].to_dict()
veh_to_route_TR = pd.read_csv('outputs/routetovehicledictFuture2018.csv')
veh_to_route_TR = veh_to_route_TR.set_index('trip_id')['route_id'].to_dict()

from_indexes = [

                '1', '49', 'K/T', 'J', 'L', 'M', 'N', 
                'BA - Blue', 'BA - Green', 'BA - Orange', 'BA - Red', 
                'BA - Yellow', 'Car', 'CarPool','TNC', 'TNC-Pool','Walk', 'Bike', 
    'CA', 'AC - others', 'SF - others','AM','AY','CC','CE','CM',
    'DE','EM','FF','GG','HF','MA','PE','RV', 'SB',
    'SC','SL','SM','SO','SR','ST','TD','UC','VC','VN','VT','WC','WH',
                'Other']
to_indexes_dict = {'CA':['CA:12867','CA:12868','CA:12869'], '1T':'AC:1-142', '49':'SF:12327', 'T':'SF:12476', 
              #'J', 'L', 'M', 'N', 
              'BA Blue':'BA:11', 
              'BA Green':'BA:5', 'BA Orange':'BA:3', 'BA Red':'BA:7', 'BA Yellow':'BA:1'}
to_indexes = ['CA', '1T', '49', 'T', 
              #'J', 'L', 'M', 'N', 
              'BA Blue', 
              'BA Green', 'BA Orange', 'BA Red', 'BA Yellow']
FT = pd.DataFrame(0, index=from_indexes, columns=to_indexes)


for veh_bs, veh_tr, project, mode_base, in_bs, in_tr in zip(list(df['Vehicles Used Baseline']), list(df['Vehicles Used TR']), list(df['Project Tried']), 
                                              list(df['Chosen Mode Baseline']), list(df['Potential INEXUS Baseline']), list(df['Potential INEXUS TR'])):
    # routes_bs = veh_to_route_bs[veh_bs]
    # routes_TR = veh_to_route_TR[veh_tr]
    # veh_bs = ','.join(veh_bs)
    # veh_tr = ','.join(veh_tr)
    

    found = 0
    
    fixed_string = veh_bs.replace(" ", ", ")
    veh_bs = ast.literal_eval(fixed_string)
    fixed_string = veh_tr.replace(" ", ", ")
    veh_tr = ast.literal_eval(fixed_string)

    routes_bs = [veh_to_route_bs.get(key, '0') for key in veh_bs]
    routes_TR = [veh_to_route_TR.get(key, '0') for key in veh_tr]
    
    attr = in_tr - in_bs
    
    BARTS = []
    if project != 'BA - Extension and Core project' :
        BARTS.append(dict_project[project])
    else:
        
        if 'BA:1' in routes_TR or 'BA:2' in routes_TR :
            BARTS.append('BA Yellow')
        if 'BA:3' in routes_TR or 'BA:4' in routes_TR :
            BARTS.append('BA Orange')
        if 'BA:5' in routes_TR or 'BA:6' in routes_TR :
            BARTS.append('BA Green')
        if 'BA:7' in routes_TR or 'BA:8' in routes_TR :
            BARTS.append('BA Red')
        if 'BA:11' in routes_TR or 'BA:12' in routes_TR :
            BARTS.append('BA Blue')
            
    for BAR in BARTS:
        
        if BAR == '1T':
            if 'AC:1-142' in routes_bs:
                FT.loc['1',BAR] += attr
                found+=1
            if any('AC:' in s for s in routes_bs) and not any('AC:' in s for s in [item for item in routes_TR if item != 'AC:1T-142']) and not any('AC:1-142' in s for s in routes_bs) :
                FT.loc['AC - others',BAR] += attr
                found+=1
        if BAR == 'CA':
            if any('CA:' in s for s in routes_bs) :
                FT.loc['CA',BAR] += attr
                found+=1
        if BAR == 'BA Yellow':
            if 'BA:1' in routes_bs:
                FT.loc['BA - Yellow',BAR] += attr
                found+=1
        if BAR == 'BA Orange':
            if 'BA:3' in routes_bs:
                FT.loc['BA - Orange',BAR] += attr
                found+=1
        if BAR == 'BA Green':
            if 'BA:5' in routes_bs:
                FT.loc['BA - Green',BAR] += attr
                found+=1
        if BAR == 'BA Red':
            if 'BA:7' in routes_bs:
                FT.loc['BA - Red',BAR] += attr
                found+=1
        if BAR == 'BA Blue':
            if 'BA:11' in routes_bs:
                FT.loc['BA - Blue',BAR] += attr
                found+=1
        if BAR == 'T':
            if 'SF:12476' in routes_bs:
                FT.loc['K/T',BAR] += attr
                found+=1
            if any('SF:' in s for s in routes_bs) and not any('SF:' in s for s in [item for item in routes_TR if item != 'SF:1001']) and not any('SF:12327' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12476' in s for s in routes_bs) and not any('SF:12477' in s for s in routes_bs) and not any('SF:12478' in s for s in routes_bs) and not any('SF:12479' in s for s in routes_bs):
                FT.loc['SF - others',BAR] += attr
                found+=1
        if BAR == '49':
            if 'SF:12327' in routes_bs:
                FT.loc['49',BAR] += attr
                found+=1
            if any('SF:' in s for s in routes_bs) and not any('SF:' in s for s in [item for item in routes_TR if item != 'SF:18608']) and not any('SF:12327' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12476' in s for s in routes_bs) and not any('SF:12477' in s for s in routes_bs) and not any('SF:12478' in s for s in routes_bs) and not any('SF:12479' in s for s in routes_bs):
                FT.loc['SF - others',BAR] += attr
                found+=1
        

        # if any('AC:' in s for s in routes_bs):
        #     FT.loc['AC',BAR] += 1
        # if any('SF:' in s for s in routes_bs):
        #     FT.loc['SF',BAR] += 1
        # if any('BA:' in s for s in routes_bs):
        #     FT.loc['BA',BAR] += 1
            
        if 'AC:1-142' in routes_bs and 'AC:1T-142' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['1',BAR] += attr
            found+=1
        if 'SF:12327' in routes_bs and 'SF:18608' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['49',BAR] += attr
            found+=1
        if 'SF:12476' in routes_bs and 'SF:1001' not in routes_TR and 'SF:1002' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['K/T',BAR] += attr
            found+=1
        if 'SF:12475' in routes_bs and 'SF:12375' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['J',BAR] += attr
            found+=1
            # if BAR == 'CA':
            #     print(routes_bs, routes_TR, attr)
        if 'SF:12477' in routes_bs and 'SF:12377' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['L',BAR] += attr
            found+=1
        if 'SF:12478' in routes_bs and 'SF:12378' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['M',BAR] += attr
            found+=1
        if 'SF:12479' in routes_bs and 'SF:12379' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['N',BAR] += attr
            found+=1
        if 'BA:1' in routes_bs and 'BA:1' not in routes_TR and 'BA:2' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['BA - Yellow',BAR] += attr
            found+=1
        if 'BA:3' in routes_bs and 'BA:3' not in routes_TR and 'BA:4' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['BA - Orange',BAR] += attr
            found+=1
        if 'BA:5' in routes_bs and 'BA:5' not in routes_TR and 'BA:6' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['BA - Green',BAR] += attr
            found+=1
        if 'BA:7' in routes_bs and 'BA:7' not in routes_TR and 'BA:8' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['BA - Red',BAR] += attr
            found+=1
        if 'BA:11' in routes_bs and 'BA:11' not in routes_TR and 'BA:12' not in routes_TR and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['BA - Blue',BAR] += attr
            found+=1
            
        if mode_base =='walk' :
            FT.loc['Walk',BAR] += attr
            found+=1
        if mode_base =='ride_hail' :
            FT.loc['TNC',BAR] += attr
            found+=1
        if mode_base =='ride_hail_pooled' :
            FT.loc['TNC-Pool',BAR] += attr
            found+=1
        if mode_base =='car' :
            FT.loc['Car',BAR] += attr
            found+=1
        if mode_base =='car_hov2' :
            FT.loc['CarPool',BAR] += attr
            found+=1
        if mode_base =='car_hov3' :
            FT.loc['CarPool',BAR] += attr
            found+=1
        if mode_base =='bike' :
            FT.loc['Bike',BAR] += attr
            found+=1

            
        if any('AC:' in s for s in routes_bs) and not any('AC:' in s for s in routes_TR) and not any('AC:1-142' in s for s in routes_bs)  and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['AC - others',BAR] += attr
            found+=1
        if any('SF:' in s for s in routes_bs) and not any('SF:' in s for s in routes_TR) and not any('SF:12327' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12475' in s for s in routes_bs) and not any('SF:12476' in s for s in routes_bs) and not any('SF:12477' in s for s in routes_bs) and not any('SF:12478' in s for s in routes_bs) and not any('SF:12479' in s for s in routes_bs) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SF - others',BAR] += attr
            found+=1
        # if any('BA:' in s for s in routes_bs) and not any('BA:' in s for s in routes_TR):
        #     FT.loc['BA',BAR] += 1
        #     found+=1
        
        if any('CA:' in s for s in routes_bs) and not any('CA:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['CA',BAR] += attr
            found+=1
        if any('AM:' in s for s in routes_bs) and not any('AM:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['AM',BAR] += attr
            found+=1
        if any('AY:' in s for s in routes_bs) and not any('AY:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['AY',BAR] += attr
            found+=1
        if any('CC:' in s for s in routes_bs) and not any('CC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['CC',BAR] += attr
            found+=1
        if any('CE:' in s for s in routes_bs) and not any('CE:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['CE',BAR] += attr
            found+=1
        if any('CM:' in s for s in routes_bs) and not any('CM:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['CM',BAR] += attr
            found+=1
        if any('DE:' in s for s in routes_bs) and not any('DE:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['DE',BAR] += attr
            found+=1
        if any('EM:' in s for s in routes_bs) and not any('EM:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['EM',BAR] += attr
            found+=1
        if any('GG:' in s for s in routes_bs) and not any('GG:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['GG',BAR] += attr
            found+=1
        if any('HF:' in s for s in routes_bs) and not any('HF:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['HF',BAR] += attr
            found+=1
        if any('MA:' in s for s in routes_bs) and not any('MA:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['MA',BAR] += attr
            found+=1
        if any('PE:' in s for s in routes_bs) and not any('PE:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['PE',BAR] += attr
            found+=1
        if any('RV:' in s for s in routes_bs) and not any('RV:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['RV',BAR] += attr
            found+=1
        if any('SB:' in s for s in routes_bs) and not any('SB:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SB',BAR] += attr
            found+=1
        if any('SC:' in s for s in routes_bs) and not any('SC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SC',BAR] += attr
            found+=1
        if any('SL:' in s for s in routes_bs) and not any('SL:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SL',BAR] += attr
            found+=1
        if any('SM:' in s for s in routes_bs) and not any('SM:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SM',BAR] += attr
            found+=1
        if any('SO:' in s for s in routes_bs) and not any('SO:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SO',BAR] += attr
            found+=1
        if any('SR:' in s for s in routes_bs) and not any('SR:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['SR',BAR] += attr
            found+=1
        if any('ST:' in s for s in routes_bs) and not any('ST:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['ST',BAR] += attr
            found+=1
        if any('TD:' in s for s in routes_bs) and not any('TD:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['TD',BAR] += attr
            found+=1
        if any('UC:' in s for s in routes_bs) and not any('UC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['UC',BAR] += attr
            found+=1
        if any('VC:' in s for s in routes_bs) and not any('VC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['VC',BAR] += attr
            found+=1
        if any('VN:' in s for s in routes_bs) and not any('VN:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['VN',BAR] += attr
            found+=1
        if any('VT:' in s for s in routes_bs) and not any('VT:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['VT',BAR] += attr
            found+=1
        if any('WC:' in s for s in routes_bs) and not any('WC:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['WC',BAR] += attr
            found+=1
        if any('WH:' in s for s in routes_bs) and not any('WH:' in s for s in routes_TR) and not any(BR  in routes_bs for BR in to_indexes_dict[BAR]):
            FT.loc['WH',BAR] += attr
            found+=1    
            
            
            
            
        if found == 0:
            FT.loc['Other',BAR] += attr  
FT.loc['Total'] = [FT['CA'].sum(),FT['1T'].sum(),FT['49'].sum(),FT['T'].sum(),FT['BA Blue'].sum(),FT['BA Green'].sum(),FT['BA Orange'].sum(),FT['BA Red'].sum(),FT['BA Yellow'].sum()]

FT = (FT/Scale).astype(int)

# ***OTHER PLOTS***

In [ ]:


grouped_sum = df.groupby('Project Tried').agg({
    'Trip Length Baseline': 'sum',
    'Trip Duration Baseline': 'sum',
    'Potential INEXUS Baseline': 'sum',
    'Trip Length TR': 'sum',
    'Trip Duration TR': 'sum',
    'Potential INEXUS TR': 'sum'
}).reset_index()

# Calculate averages for each 'Switch From' condition for "Baseline" and "TR"
conditions = [
    'Switch from another transit agency',
    'Switch from same transit agency',
    'Switch from another mode'
]
avg_incomes_baseline = [df[df['Switch From'] == condition]['Person Income'].mean() for condition in conditions]
avg_incomes_TR = [df[df['Switch From'] == condition]['Person Income'].mean() for condition in conditions]

# Display results in a table format
average_income_df = pd.DataFrame({
    'Switch From': conditions,
    'Average Household Income': avg_incomes_baseline,
    'Average Household Income': avg_incomes_TR
})

# Printing the results
print("Grouped Sums:")
print(grouped_sum)

print("\nAverage Household Incomes:")
print(average_income_df*4160)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set a custom color palette for softer colors
base_palette = sns.color_palette("colorblind")
custom_palette = sns.light_palette(base_palette[0], n_colors=len(base_palette), reverse=False, input="rgb")

# Group by 'Project Tried' and 'Switch From', then count occurrences
grouped_df = df.groupby(['Project Tried', 'Switch From']).size().unstack(fill_value=0)

# Create a new DataFrame with normalized percentages
normalized_df = grouped_df.div(grouped_df.sum(axis=1), axis=0) * 100

# Create stacked bar plot with custom color palette and labels
ax = normalized_df.plot(kind='bar', stacked=True, figsize=(18, 7), color=custom_palette)

# Set plot labels and title
# plt.xlabel('Project')
plt.ylabel('Percentage (%)', fontsize=18)
# plt.title('Percentage of "Switch From" for each project')
# plt.legend(title='Switch From', bbox_to_anchor=(1.05, 1), loc='upper left')

# Rotate x-labels for better readability
plt.xticks( ha='right')

ax.legend( bbox_to_anchor=(0.5, 1.1), loc='upper center', ncol=len(normalized_df.columns))

# Add labels on each stacked bar
for container in ax.containers:
    ax.bar_label(container, label_type='center', fontsize=18, padding=2, fmt='%.1f%%')  # Show as percentages

# ax.set_xticks(range(len(df.columns)))
ax.set_xticklabels(['1Tempo BRT', 'BART', 'Caltrain', 'Central Subway', 'Van Ness BRT'], rotation=0, fontsize=18)
ax.set_xlabel("")

    
# Adjust layout to fit everything nicely
plt.tight_layout()
plt.savefig('switch from.png', dpi=600)

# Show the plot
plt.show()


In [ ]:
# Define the mapping for aggregation
mapping = {
    'car': 'Car',
    'car_hov2': 'Car',
    'car_hov3': 'Car',
    'ride_hail': 'TNC',
    'ride_hail_pooled': 'TNC',
    'bike': 'Bike',
    'walk': 'Walk',
    'walk_transit': 'Walk',
    'drive_transit': 'Car'
}
project_name_mapping = {
    "AC - 1TEMPO": "1Tempo BRT",
    "BA - Extension and Core project": "BART",
    "CA - Electrification Project": "Caltrain",
    "SF - Central Subway": "Central Subway",
    "SF - Van Ness": "Van Ness BRT"
}

# Apply the mapping to 'Chosen Mode Baseline' column
df['Chosen Mode Baseline'] = df['Chosen Mode Baseline'].replace(mapping)
df['Project Tried'] = df['Project Tried'].replace(project_name_mapping)

# Remove rows with 'walk_transit' in 'Chosen Mode Baseline'
# df = df[df['Chosen Mode Baseline'] != 'walk_transit']

# 1. Generate the two color palettes: one blue and one orange.
custom_palette = [
    '#c9d9d3',  # very light green
    '#dadaeb',  # very light purple
    '#b2ebf2',
    '#f5c9c9',
    '#f0ebd8',  # very light yellow
    '#d0d0d0',   # another light grey
    '#e3dfd5',  # off white/greyish
    '#d9d9d9',  # light grey
]


# Ensure the palette has enough colors for the data
if len(custom_palette) < grouped_df.shape[1]:
    custom_palette = sns.color_palette("husl", n_colors=grouped_df.shape[1])  # Fallback to a diverse palette

colors = {col: custom_palette[i] for i, col in enumerate(grouped_df.columns)}

# Group by 'Project Tried' and 'Switch From', then count occurrences
grouped_df = df.groupby(['Project Tried', 'Switch From']).size().unstack(fill_value=0)

# Extract the rows in df where 'Switch From' is 'Switch from another mode'
switch_rows = df[df['Switch From'] == 'Switch from another mode']

# Group by 'Project Tried' and 'Chosen Mode Baseline' to get the count of each mode
switch_mode_grouped = switch_rows.groupby(['Project Tried', 'Chosen Mode Baseline']).size().unstack(fill_value=0)

# Replace the 'Switch from another mode' column in grouped_df with the columns from switch_mode_grouped
grouped_df = grouped_df.drop(columns='Switch from another mode')
grouped_df = pd.concat([grouped_df, switch_mode_grouped], axis=1, sort=False).fillna(0)

# Create a new DataFrame with normalized percentages
# normalized_df = grouped_df.div(grouped_df.sum(axis=1), axis=0) * 100

# Reorder the rows (i.e., the projects)
ordered_projects = ['Caltrain','1Tempo BRT', 'Van Ness BRT', 'Central Subway','BART']
grouped_df = grouped_df.loc[ordered_projects]

# Create a new DataFrame with normalized percentages
normalized_df = grouped_df.div(grouped_df.sum(axis=1), axis=0) * 100

# Plot with the custom color palette
ax = normalized_df.plot(kind='bar', stacked=True, figsize=(18, 7), color=custom_palette)



# Set plot labels and title
# plt.xlabel('Project')
plt.ylabel('Percentage (%)', fontsize=18)
# plt.title('Percentage of "Switch From" for each project')
# plt.legend(title='Switch From', bbox_to_anchor=(1.05, 1), loc='upper left')

# Rotate x-labels for better readability
plt.xticks( ha='right')

ax.legend( bbox_to_anchor=(0.5, 1.1), loc='upper center', ncol=len(normalized_df.columns), fontsize = 15)

# Add labels on each stacked bar
threshold = 3  # for example, only show numbers if the segment represents more than 5%

# Adjust bar labeling
cumulative_heights = [0] * len(normalized_df)

for container in ax.containers:
    for i, rect in enumerate(container):
        height = rect.get_height()
        x_pos = rect.get_x() + rect.get_width() / 2.
        y_pos = cumulative_heights[i] + height / 2.
        
        cumulative_heights[i] += height  # Update the cumulative height
        
        # Only label if above the threshold and height is not zero
        if height > threshold and height != 0:
            ax.text(x_pos, y_pos, f"{height:.1f}%", ha='center', va='center', fontsize=18)
            
# ax.set_xticks(range(len(df.columns)))
ax.set_xticklabels(['Caltrain','1Tempo BRT', 'Van Ness BRT',   'Central Subway','BART'], rotation=0, fontsize=18,ha='center')
ax.set_xlabel("")
# ax.set_xticks([r + rect.get_width() / 2 for r in range(len(normalized_df))])  # Adjust the x-tick positions

ax.xaxis.grid(False)
# Adjust layout to fit everything nicely
plt.tight_layout()
plt.savefig('switch from.png', dpi=600)

# Show the plot
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

project_name_mapping = {
    "AC - 1TEMPO": "1Tempo BRT",
    "BA - Extension and Core project": "BART",
    "CA - Electrification Project": "Caltrain",
    "SF - Central Subway": "Central Subway",
    "SF - Van Ness": "Van Ness BRT"
}



df['Project Tried'] = df['Project Tried'].replace(project_name_mapping)
# Set overall aesthetics
sns.set_context("talk")
sns.set(font_scale=1.2, style="whitegrid")

# Create 1x3 subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 7))

# Variables to plot
metrics = ['Diff Length', 'Diff Duration', 'Diff Potential INEXUS']
projects = ["Caltrain", "1Tempo BRT", "Van Ness BRT", "Central Subway", "BART"]
n_projects = len(projects)
labels = ['Distance', 'Duration', 'INEXUS']

mean_line_props = dict(linestyle='-', linewidth=2, color='red')
median_line_props = dict(linestyle='-', linewidth=2, color='green')

for i, (ax, metric) in enumerate(zip(axes, metrics)):
    for j, project in enumerate(projects):
        if metric == 'Diff Potential INEXUS':
            subset = df[df['Project Tried'] == project]
            ax.boxplot(subset[metric]*11.23, positions=[j], widths=0.5, patch_artist=True, showfliers=False, 
                       boxprops=dict(facecolor=sns.color_palette("pastel")[j], alpha=0.6), 
                       whis=[10, 90],  # Display whiskers for 5th and 95th percentiles
                       showmeans=True,  # Display the mean
                       meanline=True,  # Show the mean as a line
                       meanprops=mean_line_props,  # Custom properties for mean line
                       medianprops=median_line_props)  # Custom properties for median line
        else:
            subset = df[df['Project Tried'] == project]
            ax.boxplot(subset[metric], positions=[j], widths=0.5, patch_artist=True, showfliers=False, 
                       boxprops=dict(facecolor=sns.color_palette("pastel")[j], alpha=0.6),
                       whis=[10, 90],  # Display whiskers for 5th and 95th percentiles
                       showmeans=True,  # Display the mean
                       meanline=True,  # Show the mean as a line
                       meanprops=mean_line_props,  # Custom properties for mean line
                       medianprops=median_line_props)  # Custom properties for median line

    
    # Setting y-label for each subplot
    if i ==0:
        ax.set_ylabel(f'Absolute Change', fontsize=18)
    
    # X-axis settings
    ax.set_xticks([n_projects/2])  # centering the metric name
    ax.set_xticklabels([labels[i]], fontsize=18)  # using the first word of the metric name, and setting fontsize to 18
    ax.xaxis.grid(False)  # <-- Set visible to False
    ax.yaxis.grid(True, linestyle='--', which='major', color='gray', alpha=.25)
    ax.tick_params(axis='both', which='major', labelsize=18)  # setting the tick font size to 18

# Global title
# fig.suptitle('Absolute Change and Distribution from Baseline', fontsize=18, y=1.08)

# Global legend within the plots, horizontally over the plots
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=sns.color_palette("pastel")[i], edgecolor='gray', label=project) 
                   for i, project in enumerate(projects)]
fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, 0.02), ncol=len(projects), fontsize=18)

plt.tight_layout()
plt.savefig('column_plot_distribution.png', dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
metrics = ['Distance','Duration','INEXUS']
metrics_bs = ['Trip Length Baseline','Trip Duration Baseline','Potential INEXUS Baseline']
metrics_tr = ['Trip Length TR','Trip Duration TR','Potential INEXUS TR']
labels = ['Distance', 'Duration', 'INEXUS']

# Calculate percentage change for each metric and project
percentage_changes = {}

for mb, mt, m in zip(metrics_bs, metrics_tr, metrics):
    percentage_changes[m] = []
    for project in projects:

        baseline_sum = df[df['Project Tried'] == project][mb].sum()
        tr_sum = df[df['Project Tried'] == project][mt].sum()

        change = 100 * (tr_sum - baseline_sum) / baseline_sum  # percentage change formula
        percentage_changes[m].append(change)

# Plotting
# Assuming the percentage change for each metric and project is already calculated as per previous code in `percentage_changes`
# We will adjust the plotting structure:

fig, ax = plt.subplots(figsize=(18, 7))
width = 0.15  # adjust as needed
ind = np.arange(len(metrics))  # the x locations for the metrics

# Loop to plot bars for each project under each metric
for i, project in enumerate(projects):
    
    values = [percentage_changes[metric][i]  if metric != 'INEXUS' else -percentage_changes[metric][i] for metric in metrics ]


    bars = ax.bar(ind + i*width, values, width, color=sns.color_palette("pastel")[i], label=project, alpha=0.6)
    
    
    # Adding values on top or below the bars based on value's sign
    for bar in bars:
        height = bar.get_height()
        if height >= 0:
            ax.text(bar.get_x() + bar.get_width()/2., 1.01*height, f'{height:.1f}%', 
                    ha='center', va='bottom', fontsize=18)
        else:
            ax.text(bar.get_x() + bar.get_width()/2., 1.05*height, f'{height:.1f}%', 
                    ha='center', va='top', fontsize=18)

# Adjust x-axis labels and other aesthetics
ax.set_ylabel('Percentage Change', fontsize=18)
ax.set_xticks(ind + width * (len(projects) / 2))
ax.set_xticklabels(labels, fontsize=18)
# Put the legend inside the graph, the loc parameter can be adjusted to place the legend at a different position
ax.legend(fontsize=18, loc='lower right')
ax.grid(True, axis='y', linestyle='--', which='major', color='gray', alpha=.25)
ax.set_ylim(-12, 12)# Adjust the upper limit as necessary
plt.tight_layout()
ax.xaxis.grid(False)
plt.savefig('percentage_change_grouped3.png', dpi=600, bbox_inches='tight')
plt.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Sample dataframe (replace this with your actual dataframe)
# df = ...
df = df[df['Person Race']!='American Indian and Alaska Native tribes specified; or American Indian or Alaska Native, not specified and no other races']
# Compute the average income per project
avg_income_per_project = df.groupby('Project Tried')['Person Income'].mean()

# # Compute the proportion of each race per project
race_proportions = df.groupby(['Project Tried', 'Person Race']).size().unstack().fillna(0)
race_proportions_percentage = (race_proportions.divide(race_proportions.sum(axis=1), axis=0) * 100).fillna(0)
race_proportions = race_proportions.divide(race_proportions.sum(axis=1), axis=0)
print(avg_income_per_project)

ordered_projects = ['Caltrain','1Tempo BRT', 'Van Ness BRT', 'Central Subway','BART']

# Order the dataframes according to the specified order of projects
avg_income_per_project = avg_income_per_project.loc[ordered_projects]
race_proportions = race_proportions.loc[ordered_projects]
race_proportions_percentage = race_proportions_percentage.loc[ordered_projects]



# Compute the height of each segment of the bar
segment_heights = race_proportions.multiply(avg_income_per_project, axis=0)

# Light colors using the Pastel1 colormap
colors = plt.cm.Pastel2.colors

# Plotting
ax = segment_heights.plot(kind='bar', stacked=True, figsize=(18, 7), color=colors)

# Annotate the bars with the average income values
for idx, value in enumerate(avg_income_per_project):
    ax.text(idx, value, f'{value:,.0f}', ha='center', va='bottom', fontsize=18)

# Annotate with percentage values inside each sub-bar
cum_heights = segment_heights.cumsum(axis=1)
for col in race_proportions_percentage.columns:
    for idx, value in enumerate(race_proportions_percentage[col]):
        if value > 3:  # Only annotate if value is significant enough for visibility
            height = cum_heights.at[segment_heights.index[idx], col] - 0.5 * segment_heights.at[segment_heights.index[idx], col]
            ax.text(idx, height, f'{value:.0f}%', ha='center', va='center', fontsize=18, color='black')  # using black for visibility against light colors

ax.set_ylabel('Average Person Income', fontsize = 16)
# ax.set_title('Average Person Income by Project, Split by Race')

# Replace x labels as specified
x_labels = ["Caltrain", "1Tempo BRT", "Van Ness BRT", "Central Subway", "BART"]
ax.set_xticks(range(len(x_labels)))
ax.set_xticklabels(x_labels, rotation=0, ha="center", fontsize = 18)
ax.set_xlabel("")


original_labels = ['Alaska Native alone', 
                   'American Indian alone', 
                   'Asian alone', 
                   'Black or African American alone', 
                   'White alone', 
                  'American Indian and Alaska Native tribes specified; or American Indian or Alaska Native, not specified and no other races', 
                  'Native Hawaiian and Other Pacific Islander alone',
                   'Some Other Race alone'
                  ]  # Assuming these are your original labels
summarized_labels = ['Alaska Native', 'American Indian', 'Asian', 
                     'Black or African American', 'White', 'Other', 'Native Hawaiian', 'Other'
                    ]  # Shorter labels
overall_avg_income = df['Person Income'].mean()
ax.axhline(y=overall_avg_income, color='black', linestyle='--', linewidth=2)
def currency_formatter(x, pos):
    return f"${x:,.0f}"

ax.yaxis.set_major_formatter(FuncFormatter(currency_formatter))

legend_dict = dict(zip(original_labels, summarized_labels))
handles, labels = ax.get_legend_handles_labels()
new_labels = [legend_dict[label] if label in legend_dict else label for label in labels]
ax.legend(handles, new_labels, title='Race', fontsize=14, bbox_to_anchor=(0.30, 0.99), loc='upper center')
ax.xaxis.grid(False)

plt.savefig('income2.png', dpi=600, bbox_inches='tight')
plt.tight_layout()
plt.show()


In [ ]:
 df['Person Race'].value_counts()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# Sample dataframe (replace this with your actual dataframe)
# df = ...
df = df[df['Person Race']!='American Indian and Alaska Native tribes specified; or American Indian or Alaska Native, not specified and no other races']

# Compute the average income per project
avg_income_per_project = df.groupby('Project Tried')['Person Income'].mean()

ordered_projects = ['Caltrain','1Tempo BRT', 'Van Ness BRT', 'Central Subway','BART']

# Order the dataframes according to the specified order of projects
avg_income_per_project = avg_income_per_project.loc[ordered_projects]

# Plotting
fig, ax = plt.subplots(figsize=(18, 7))
avg_income_per_project.plot(kind='bar', ax=ax, color='skyblue')

# Annotate the bars with the average income values
for idx, value in enumerate(avg_income_per_project):
    ax.text(idx, value + 1000, f'${value:,.0f}', ha='center', va='bottom', fontsize=18) # adding +1000 for some space above the bar

ax.set_ylabel('Average Person Income', fontsize=16)
ax.set_title('Average Person Income by Project', fontsize=20)

x_labels = ["Caltrain", "1Tempo BRT", "Van Ness BRT", "Central Subway", "BART"]
ax.set_xticks(range(len(x_labels)))
ax.set_xticklabels(x_labels, rotation=0, ha="center", fontsize=18)

overall_avg_income = df['Person Income'].mean()
ax.axhline(y=overall_avg_income, color='black', linestyle='--', linewidth=2)

def currency_formatter(x, pos):
    return f"${x:,.0f}"

ax.yaxis.set_major_formatter(FuncFormatter(currency_formatter))
ax.xaxis.grid(False)

plt.savefig('income2.png', dpi=600, bbox_inches='tight')
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np

df = df[df['Person Race']!='American Indian and Alaska Native tribes specified; or American Indian or Alaska Native, not specified and no other races']
dfx = df[df['HS Income']>0]
# Compute the average income per project
# avg_income_per_project = dfx.groupby('Project Tried')['HS Income'].mean()
avg_income_per_project_switch_test = dfx.pivot_table(values='HS Income', index='Project Tried', columns='Switch From', aggfunc='mean')


ordered_projects = ['Caltrain','1Tempo BRT', 'Van Ness BRT', 'Central Subway','BART']

# Order the dataframes according to the specified order of projects
avg_income_per_project_switch_test = avg_income_per_project_switch_test.loc[ordered_projects]
print(avg_income_per_project)

fig, ax1 = plt.subplots(figsize=(15, 7))

# Bar width for grouped bars
bar_width = 0.2
switch_categories = df['Switch From'].unique()
num_switches = len(switch_categories)
positions = list(range(len(ordered_projects)))

# Pastel1 palette
colors = plt.cm.Pastel1(np.linspace(0, 1, num_switches))

for index, (switch, color) in enumerate(zip(switch_categories, colors)):
    switch_values = avg_income_per_project_switch_test[switch]
    ax1.bar([pos + index * bar_width for pos in positions], switch_values, width=bar_width, label=switch, color=color)

# Annotate bars with their values
for index, project in enumerate(ordered_projects):
    for switch_index, switch in enumerate(switch_categories):
        value = avg_income_per_project_switch_test.at[project, switch]
        ax1.text(index + switch_index * bar_width, value + 1000, f'${value:,.0f}', ha='center', va='bottom', fontsize=10)

# Adjust x-axis to accommodate grouped bars
ax1.set_xticks([pos + bar_width for pos in positions])
ax1.set_xticklabels(ordered_projects, rotation=0, fontsize=10)

# Overall average line, legend placement and formatting
ax1.axhline(y=105854, color='gray', linestyle='--', linewidth=2)
ax1.legend( loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=len(switch_categories))  # Adjusted these parameters
ax1.set_ylabel('Average Household Income', fontsize=16)
ax1.yaxis.set_major_formatter(FuncFormatter(currency_formatter))
ax1.xaxis.grid(False)

plt.tight_layout()
plt.savefig('average_income_grouped.png', dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
def swap_columns_for_switches(df, switch_categories):
    for project in ordered_projects:
        first, third = df.xs((project, switch_categories[0])), df.xs((project, switch_categories[2]))
        df.loc[(project, switch_categories[0])], df.loc[(project, switch_categories[2])] = third.values, first.values
    return df

grouped_race_percentages = swap_columns_for_switches(grouped_race_percentages, df['Switch From'].unique())
df = df[df['Person Race'] != 'American Indian and Alaska Native tribes specified; or American Indian or Alaska Native, not specified and no other races']

avg_income_per_project = df.groupby('Project Tried')['Person Income'].mean().loc[ordered_projects]
race_proportions_percentage = (df.groupby(['Project Tried', 'Person Race']).size().unstack().fillna(0).divide(df.groupby(['Project Tried', 'Person Race']).size().unstack().sum(axis=1), axis=0) * 100).loc[ordered_projects]

fig2, ax2 = plt.subplots(figsize=(15, 7))

bar_width, switch_categories, positions = 0.2, df['Switch From'].unique(), list(range(len(ordered_projects)))

def add_labels(data, ax, shift=0):
    for idx, project in enumerate(ordered_projects):
        cum_height = 0
        for col_num, race in enumerate(data.columns):
            value, y_position = data.loc[project, race], cum_height + 0.5 * data.loc[project, race]
            if value > 5:
                ax.text(idx + shift, y_position, f"{value:.0f}%", ha="center", va="center", fontsize=12, color='black')
            cum_height += value

for idx, project in enumerate(ordered_projects):
    for switch_index, switch in enumerate(switch_categories):
        ax2.text(idx + bar_width * (switch_index - 1) + bar_width / 2, 102, custom_titles.get(switch, switch), ha="center", fontsize=12, fontweight='bold')

for switch_index, switch in enumerate(switch_categories):
    grouped_race_percentages.xs(switch, level='Switch From').reindex(ordered_projects).fillna(0).plot(kind='bar', stacked=True, ax=ax2, width=bar_width, color=colors, position=switch_index - 1)
    add_labels(grouped_race_percentages.xs(switch, level='Switch From').reindex(ordered_projects).fillna(0), ax2, shift=bar_width * (switch_index - 1) + bar_width / 2)

ax2.set_xlim(-1.5 * bar_width, len(ordered_projects) - 1.5 * bar_width)
ax2.set_xticks(positions)
ax2.set_xticklabels(ordered_projects, rotation=0, ha="center", fontsize=13)
ax2.legend(handles[:len(unique_labels)], [legend_dict[label] if label in legend_dict else label for label in grouped_race_proportions.columns], fontsize=12, loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=4)
ax2.yaxis.set_major_formatter(FuncFormatter(percent_formatter))
ax2.set_xlabel("")
plt.tight_layout()
plt.savefig('race_proportions_split_corrected.png', dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
grouped = df.groupby('Project Tried')

vmt_baseline = grouped['Trip Length Baseline'].sum()
vmt_TR = grouped['Trip Length TR'].sum()

vht_baseline = grouped['Trip Duration Baseline'].sum()
vht_TR = grouped['Trip Duration TR'].sum()

plan_mode_baseline = grouped['Planned Mode Baseline'].value_counts()
plan_mode_TR = grouped['Planned Mode TR'].value_counts()

mode_baseline = grouped['Chosen Mode Baseline'].value_counts()
mode_TR = grouped['Chosen Mode TR'].value_counts()

inexus_baseline = grouped['Potential INEXUS Baseline'].sum()
inexus_TR = grouped['Potential INEXUS TR'].sum()


In [ ]:
vmt_baseline

In [ ]:
vmt_TR

In [ ]:
vht_baseline

In [ ]:
vht_TR

In [ ]:
plan_mode_baseline

In [ ]:
plan_mode_TR

In [ ]:
mode_baseline

In [ ]:
mode_TR

In [ ]:
inexus_baseline

In [ ]:
inexus_TR

In [ ]:
TR = inexus_TR
BS = inexus_baseline

index_union = BS.index.union(TR.index)
baseline_reindexed = BS.reindex(index_union, fill_value=0)
TR_reindexed = TR.reindex(index_union, fill_value=0)

# Calculate the difference between the two reindexed pivot tables
difference_table = (TR_reindexed - baseline_reindexed)/(-baseline_reindexed)*100

# The `difference_table` will contain the differences between the two original pivot tables.
print(difference_table)

In [ ]:
TR = mode_TR
BS = mode_baseline

index_union = BS.index.union(TR.index)
baseline_reindexed = BS.reindex(index_union, fill_value=0)
TR_reindexed = TR.reindex(index_union, fill_value=0)

# Calculate the difference between the two reindexed pivot tables
difference_table = TR_reindexed - baseline_reindexed 

# The `difference_table` will contain the differences between the two original pivot tables.
print(difference_table)

# ***EVENTS STATS PUBLIC TRANSIT***

In [11]:
Dir_beam = 'gs://beam-core-outputs/sfbay-baseline2018-30pct-20230825/beam/year-2018-iteration-10/ITERS/it.0'
Dir_beam_TR = 'gs://beam-core-outputs/sfbay-tr-30pct-20231014/beam/year-2018-iteration-10/ITERS/it.0'
Scale = 0.3

Events = pd.read_csv(Dir_beam+'/0.events.csv.gz', nrows = None, 
                     usecols=['type', 'numPassengers', 'capacity', 'mode', 'vehicle', 'length', 'duration','arrivalTime','departureTime', 'primaryFuel', 'secondaryFuel']
                    )
Events = Events[(Events['type']=='PathTraversal')&(Events['mode'].isin(['bus','tram','subway','cable_car','rail', 'ferry']))]
agencies = []

for vehicle in Events['vehicle']:
    agencies.append(vehicle.split(':')[0])
Events['agency'] = agencies

vehicle_to_routes = pd.read_csv('outputs/routetovehicledictBaseline.csv')
vehicle_to_routes = vehicle_to_routes.set_index('0')['1'].to_dict()
Events['route_id'] = Events['vehicle'].map(vehicle_to_routes)
print(len(Events))
Events = Events[(~Events['route_id'].str.contains('BA:19'))]
print(len(Events))


Events['numPassengers'] = Events['numPassengers']/Scale
Events['capacity'] = Events['capacity']/Scale

Events['numPassengers'] = Events['numPassengers'].astype(int)
Events['capacity'] = Events['capacity'].astype(int)

# Events = Events.dropna(axis=1, how='all')
transit_agencies = {
    'AC': 'Alameda-Contra Costa Transit District (AC Transit)',
    # 'AM': 'Capitol Corridor',    
    # 'AY': 'American Canyon Transit',
    'BA': 'BART',
    # 'CC': 'County Connection (Central Contra Costa Transit Authority)',
    # 'CE': 'ACE Altamont Corridor Express',
    # 'CM': 'San Mateo County Transit',
    # 'DE': 'Dumbarton Express',
    # 'EM': 'Emery Go-Round',
    # 'FF': 'Fairfield and Suisun Transit (FAST)',
    # 'GG': 'Golden Gate Transit',
    # 'HF': 'Alcatraz Hornblower Ferry',
    # 'MA': 'Marin Transit',
    # 'SB': 'San Francisco Bay Ferry',
    # 'SC': 'Santa Cruz Metro',
    'SF': 'San Francisco Municipal Transportation Agency (SFMTA)',
    # 'SL': 'SolTrans',
    # 'SM': 'Stanford Marguerite Shuttle',
    # 'SO': 'Sonoma County Transit',
    # 'SR': 'Santa Rosa CityBus',
    # 'ST': 'SamTrans',
    # 'TD': 'Tri Delta Transit',
    # 'VC': 'Vacaville City Coach',
    # 'VN': 'Vine Transit (Napa County)',
    # 'VT': 'Santa Clara Valley Transportation Authority (VTA)',
    # 'WH': 'Livermore Amador Valley Transit Authority',
    'caltrain-ca-us': 'Caltrain',
    # 'petalumatransit-petaluma-ca-us': 'Petaluma Transit',
    # 'riovista-ca-us': 'Rio Vista Delta Breeze',
    # 'unioncity-ca-us': 'Union City Transit',
    # 'westcat-ca-us': 'Western Contra Costa Transit Authority (WestCAT)'
}

vehicle_ids = {
    'SF:12327':'Line 49 Van Ness/Mission' ,
    'AC:1-142':'1 AC' ,
    # 'CA:12867':'Caltrain Bullet, Baby Bullet' ,
    # 'CA:12867':'Caltrain Limitåd' ,
    # 'CA:12867':'Caltrain Local' ,
    'SF:12475':'Muni Metro J' ,
    'SF:12476':'Muni Metro K/T' ,
    'SF:12311':'LINE 30 Stockton' ,
    # 'SF:1106':'Muni Metro T' ,
    # 'SF:1102':'Muni Metro K' ,
    'SF:12477':'Muni Metro L' ,
    'SF:12478':'Muni Metro M' ,
    'SF:12479':'Muni Metro N' ,
    'BA:1':'BA Yellow' ,
    # 'BA:2':'BA Yellow 2',
    'BA:3':'BA Orange',
    # 'BA:4':'BA Orange 4',
    'BA:5':'BA Green',
    # 'BA:6':'BA Green 6',
    'BA:7':'BA Red',
    # 'BA:8':'BA Red 8',
    # 'BA:10':'BA Blue',
    'BA:11':'BA Blue',
    ':':'All',
}
n_trips = []
names = []
pass_kms =  []
supp_kms =  []
modes = []
lengths = []
pass_durations = []
energies = []
durations = []

for agency in transit_agencies.keys():
    names.append(transit_agencies[agency])
    events_agency = Events[Events['agency']==agency]
    n_trips.append(len(events_agency['vehicle'].value_counts()))
    pass_kms.append((events_agency['numPassengers'] * events_agency['length']).sum())
    supp_kms.append((events_agency['capacity'] * events_agency['length']).sum())
    modes.append(list(np.unique(events_agency['mode'])))
    lengths.append(int(np.sum(events_agency['length'])))
    pass_durations.append((events_agency['numPassengers'] *(events_agency['arrivalTime'] - events_agency['departureTime'])).sum())
    energies.append((events_agency['primaryFuel'] + events_agency['secondaryFuel']).sum())
    durations.append((events_agency['arrivalTime'] - events_agency['departureTime']).sum())
    
for route in vehicle_ids.keys():
    names.append(vehicle_ids[route])
    if route != ':':
        events_route = Events[Events['route_id']==route]
    else:
        events_route = Events    
    n_trips.append(len(events_route['vehicle'].value_counts()))
    pass_kms.append((events_route['numPassengers'] * events_route['length']).sum())
    supp_kms.append((events_route['capacity'] * events_route['length']).sum())
    modes.append(list(np.unique(events_route['mode'])))
    lengths.append(int(np.sum(events_route['length'])))
    pass_durations.append((events_route['numPassengers'] *(events_route['arrivalTime'] - events_route['departureTime'])).sum())
    energies.append((events_route['primaryFuel'] + events_route['secondaryFuel']).sum())
    durations.append((events_route['arrivalTime'] - events_route['departureTime']).sum())
    
Events = pd.read_csv(Dir_beam_TR+'/0.events.csv.gz', nrows = None, 
                     usecols=['type', 'numPassengers', 'capacity', 'mode', 'vehicle', 'length', 'duration','arrivalTime','departureTime', 'primaryFuel', 'secondaryFuel']
                    )
Events = Events[(Events['type']=='PathTraversal')&(Events['mode'].isin(['bus','tram','subway','cable_car','rail', 'ferry']))]
agencies = []

vehicle_to_routes = pd.read_csv('outputs/routetovehicledictTR.csv')
vehicle_to_routes = vehicle_to_routes.set_index('0')['1'].to_dict()
Events['route_id'] = Events['vehicle'].map(vehicle_to_routes)

for vehicle in Events['vehicle']:
    agencies.append(vehicle.split(':')[0])
Events['agency'] = agencies

Events['numPassengers'] = Events['numPassengers']/3*10
Events['capacity'] = Events['capacity']/3*10

Events['numPassengers'] = Events['numPassengers'].astype(int)
Events['capacity'] = Events['capacity'].astype(int)

transit_agencies = {
    'AC': 'Alameda-Contra Costa Transit District (AC Transit) - TR',
    # 'AM': 'Capitol Corridor',    
    # 'AY': 'American Canyon Transit',
    'BA': 'BART - TR',
    # 'CC': 'County Connection (Central Contra Costa Transit Authority)',
    # 'CE': 'ACE Altamont Corridor Express',
    # 'CM': 'San Mateo County Transit',
    # 'DE': 'Dumbarton Express',
    # 'EM': 'Emery Go-Round',
    # 'FF': 'Fairfield and Suisun Transit (FAST)',
    # 'GG': 'Golden Gate Transit',
    # 'HF': 'Alcatraz Hornblower Ferry',
    # 'MA': 'Marin Transit',
    # 'SB': 'San Francisco Bay Ferry',
    # 'SC': 'Santa Cruz Metro',
    'SF': 'San Francisco Municipal Transportation Agency (SFMTA) - TR',
    # 'SL': 'SolTrans',
    # 'SM': 'Stanford Marguerite Shuttle',
    # 'SO': 'Sonoma County Transit',
    # 'SR': 'Santa Rosa CityBus',
    # 'ST': 'SamTrans',
    # 'TD': 'Tri Delta Transit',
    # 'VC': 'Vacaville City Coach',
    # 'VN': 'Vine Transit (Napa County)',
    # 'VT': 'Santa Clara Valley Transportation Authority (VTA)',
    # 'WH': 'Livermore Amador Valley Transit Authority',
    'Caltrain': 'Caltrain - TR',
    # 'petalumatransit-petaluma-ca-us': 'Petaluma Transit',
    # 'riovista-ca-us': 'Rio Vista Delta Breeze',
    # 'unioncity-ca-us': 'Union City Transit',
    # 'westcat-ca-us': 'Western Contra Costa Transit Authority (WestCAT)'
}

vehicle_ids = {
    'SF:18608':'Line 49 Van Ness/Mission - TR' ,
    'AC:1T-142':'1Tempo AC' ,
    # 'CA:12867':'Caltrain Bullet, Baby Bullet - TR' ,
    # 'CA:12867':'Caltrain Limited - TR' ,
    #  'CA:12867':'Caltrain Local - TR',
    'SF:12475':'Muni Metro J - TR' ,
    'SF:1001':'Muni Metro T - TR' ,
    'SF:1002':'Muni Metro K - TR' ,
    'SF:12311':'LINE 30 Stockton - TR' ,
    'SF:12477':'Muni Metro L - TR' ,
    'SF:12478':'Muni Metro M - TR' ,
    'SF:12479':'Muni Metro N - TR' ,
    'BA:1':'BA Yellow 1 - TR' ,
    'BA:2':'BA Yellow 2 - TR',
    'BA:3':'BA Orange 3 - TR',
    'BA:4':'BA Orange 4 - TR',
    'BA:5':'BA Green 5 - TR',
    'BA:6':'BA Green 6 - TR',
    'BA:7':'BA Red 7 - TR',
    'BA:8':'BA Red 8 - TR',
    'BA:11':'BA Blue 11 - TR',
    'BA:12':'BA Blue 12 - TR',
    ':':'All - TR',
}

for agency in transit_agencies.keys():
    names.append(transit_agencies[agency])
    events_agency = Events[Events['agency']==agency]
    n_trips.append(len(events_agency['vehicle'].value_counts()))
    pass_kms.append((events_agency['numPassengers'] * events_agency['length']).sum())
    supp_kms.append((events_agency['capacity'] * events_agency['length']).sum())
    modes.append(list(np.unique(events_agency['mode'])))
    lengths.append(int(np.sum(events_agency['length'])))
    pass_durations.append((events_agency['numPassengers'] *(events_agency['arrivalTime'] - events_agency['departureTime'])).sum())
    energies.append((events_agency['primaryFuel'] + events_agency['secondaryFuel']).sum())
    durations.append((events_agency['arrivalTime'] - events_agency['departureTime']).sum())

for route in vehicle_ids.keys():
    names.append(vehicle_ids[route])
    if route != ':':
        events_route = Events[Events['route_id']==route]
    else:
        events_route = Events
    n_trips.append(len(events_route['vehicle'].value_counts()))
    pass_kms.append((events_route['numPassengers'] * events_route['length']).sum())
    supp_kms.append((events_route['capacity'] * events_route['length']).sum())
    modes.append(list(np.unique(events_route['mode'])))
    lengths.append(int(np.sum(events_route['length'])))
    pass_durations.append((events_route['numPassengers'] *(events_route['arrivalTime'] - events_route['departureTime'])).sum())
    energies.append((events_route['primaryFuel'] + events_route['secondaryFuel']).sum())
    durations.append((events_route['arrivalTime'] - events_route['departureTime']).sum())
    
Table = pd.DataFrame()
Table.index = names
Table['Number of Trips per Direction'] = n_trips
Table['VMT km M'] = np.array(lengths)/1000000
Table['VHT km M'] = np.array(durations)/1000000
Table['PHT M'] = np.array(pass_durations)/1000000000
# Table['Energy GJ'] = np.array(energies)/1000000000
Table['RPK km M'] = np.array(pass_kms)/1000000000
Table['ASK km M'] = np.array(supp_kms)/1000000000
# Table['Modes'] = modes

# Table.loc['BART'] = Table.loc['BA Yellow'] + Table.loc['BA Green'] + Table.loc['BA Red'] + Table.loc['BA Blue'] + Table.loc['BA Orange']
Table.loc['BA Blue - TR'] = Table.loc['BA Blue 11 - TR'] + Table.loc['BA Blue 12 - TR']
Table.loc['BA Yellow - TR'] = Table.loc['BA Yellow 1 - TR'] + Table.loc['BA Yellow 2 - TR']
Table.loc['BA Orange - TR'] = Table.loc['BA Orange 3 - TR'] + Table.loc['BA Orange 4 - TR']
Table.loc['BA Green - TR'] = Table.loc['BA Green 5 - TR'] + Table.loc['BA Green 6 - TR']
Table.loc['BA Red - TR'] = Table.loc['BA Red 7 - TR'] + Table.loc['BA Red 8 - TR']
# Table.loc['Caltrain'] = Table.loc['Caltrain Bullet, Baby Bullet'] + Table.loc['Caltrain Limited'] + Table.loc['Caltrain Local']
# Table.loc['Caltrain - TR'] = Table.loc['Caltrain Bullet, Baby Bullet - TR'] + Table.loc['Caltrain Limited - TR'] + Table.loc['Caltrain Local - TR']
Table.loc['BA-CA-SF-AC'] = Table.loc['Caltrain'] + Table.loc['BART'] +Table.loc['Alameda-Contra Costa Transit District (AC Transit)'] + Table.loc['San Francisco Municipal Transportation Agency (SFMTA)']
Table.loc['BA-CA-SF-AC - TR'] = Table.loc['Caltrain - TR'] + Table.loc['BART - TR'] +Table.loc['Alameda-Contra Costa Transit District (AC Transit) - TR'] + Table.loc['San Francisco Municipal Transportation Agency (SFMTA) - TR']
Table.loc['Other'] = Table.loc['All'] - Table.loc['BA-CA-SF-AC'] 
Table.loc['Other - TR'] = Table.loc['All - TR'] - Table.loc['BA-CA-SF-AC - TR'] 
Table.loc['Muni Metro K/T - TR'] = Table.loc['Muni Metro K - TR'] + Table.loc['Muni Metro T - TR'] 
Table.loc['Muni Metro'] = Table.loc['Muni Metro K/T'] + Table.loc['Muni Metro J']  + Table.loc['Muni Metro L']  + Table.loc['Muni Metro M']  + Table.loc['Muni Metro N'] 
Table.loc['Muni Metro - TR'] = Table.loc['Muni Metro K/T - TR'] + Table.loc['Muni Metro J - TR']  + Table.loc['Muni Metro L - TR']  + Table.loc['Muni Metro M - TR']  + Table.loc['Muni Metro N - TR'] 

Table['AV Load Factor'] = Table['RPK km M']/Table['ASK km M']
Table['AV Speed vehicles km/h'] = Table['VMT km M']/Table['VHT km M']
Table['AV Speed pass km/h'] = Table['RPK km M']/Table['PHT M']

Table = Table.drop('BA Yellow 1 - TR')
Table = Table.drop('BA Yellow 2 - TR')
Table = Table.drop('BA Orange 3 - TR')
Table = Table.drop('BA Orange 4 - TR')
Table = Table.drop('BA Green 5 - TR')
Table = Table.drop('BA Green 6 - TR')
Table = Table.drop('BA Red 7 - TR')
Table = Table.drop('BA Red 8 - TR')
Table = Table.drop('BA Blue 12 - TR')
Table = Table.drop('BA Blue 11 - TR')
# Table = Table.drop('Caltrain Bullet, Baby Bullet - TR')
# Table = Table.drop('Caltrain Limited - TR')
# Table = Table.drop('Caltrain Local - TR')
# Table = Table.drop('Caltrain Bullet, Baby Bullet')
# Table = Table.drop('Caltrain Limited')
# Table = Table.drop('Caltrain Local')


In [13]:
Table

,Number of Trips per Direction,VMT km M,VHT km M,PHT M,RPK km M,ASK km M,AV Load Factor,AV Speed vehicles km/h,AV Speed pass km/h
Alameda-Contra Costa Transit District (AC Transit),6621.0,119.850094,17.702160,0.245189,1.756186,7.910106,0.222018,6.770366,7.162568
BART,741.0,46.085164,3.049020,0.379938,5.150379,64.657485,0.079656,15.114746,13.555846
San Francisco Municipal Transportation Agency (SFMTA),11881.0,155.181355,29.966602,0.336297,1.754327,10.803367,0.162387,5.178477,5.216605
Caltrain,92.0,7.080731,0.491040,0.022179,0.322154,23.600079,0.013651,14.419866,14.525007
Line 49 Van Ness/Mission,236.0,2.925226,0.827820,0.006815,0.023264,0.193065,0.120501,3.533650,3.413662
1 AC,239.0,3.748890,0.778380,0.008083,0.038994,0.247427,0.157599,4.816272,4.824310
Muni Metro J,220.0,2.246090,0.474120,0.004422,0.021574,0.208886,0.103280,4.737387,4.878318
Muni Metro K/T,297.0,5.428144,1.115880,0.020658,0.112928,0.504817,0.223700,4.864451,5.466523
LINE 30 Stockton,424.0,3.060325,0.977100,0.004711,0.014388,0.201981,0.071235,3.132049,3.054305
Muni Metro L,244.0,3.023406,0.519840,0.018279,0.112255,0.281177,0.399233,5.816032,6.141364


In [17]:
RR = pd.read_csv('outputs/route_ridershipNewit5.csv', index_col = 0)
RR = RR[(~RR.index.str.contains('BA:19'))]

columns = ['AG', 'Line', 'Case', 'Ridership', '# Trips', 'VMT [M km]', 'VHT [M h]', 
           'RPK [M km]', 'PHT [M h]', 'ASK [M km]', 'Load Factor', 
           'Av Sp. Veh. [km/h]', 'Av Sp. Pas. [km/h]' ]

agencies =  ['CA', 'CA', 
             'AC', 'AC', 'AC', 'AC',
             'BA', 'BA', 'BA', 'BA', 'BA', 'BA', 'BA', 'BA', 'BA', 'BA', 'BA', 'BA',
             'SF', 'SF', 'SF', 'SF', 'SF', 'SF', 'SF', 'SF', 'SF', 'SF', 'SF', 'SF', 'SF', 'SF', 'SF', 'SF', 'SF', 'SF',  
             'All', 'All', 'BA-CA-SF-AC', 'BA-CA-SF-AC', 'Other','Other'
            ]
lines = ['All', 'All',
         'All', 'All', '1', '1T',
         'All', 'All', 'Blue', 'Blue', 'Green', 'Green', 'Orange', 'Orange', 'Red', 'Red', 'Yellow', 'Yellow',
         'All', 'All', '49', '49', 'K/T', 'K/T', 'K', 'T', 'J', 'J', 'L', 'L', 'M', 'M', 'N', 'N', 'Metro', 'Metro',
          'All', 'All','BA-CA-SF-AC', 'BA-CA-SF-AC', 'Other','Other'
        ]
cases = ['BAU', 'TR', 
         'BAU', 'TR', 'BAU', 'TR', 
         'BAU', 'TR', 'BAU', 'TR', 'BAU', 'TR', 'BAU', 'TR', 'BAU', 'TR', 'BAU', 'TR', 
         'BAU', 'TR', 'BAU', 'TR', 'BAU', 'TR', 'TR', 'TR', 'BAU', 'TR', 'BAU', 'TR', 'BAU', 'TR', 'BAU', 'TR', 'BAU', 'TR'
         , 'BAU', 'TR', 'BAU', 'TR','BAU', 'TR'
        ]
         
rrs = [['CA:'], ['CA:'],
      ['AC:'], ['AC:'], ['AC:1-142'], ['AC:1T-142'],
      ['BA:'], ['BA:'], ['BA:11'], ['BA:11', 'BA:12'], ['BA:5'], ['BA:5', 'BA:6'], ['BA:3'], ['BA:3', 'BA:4'], ['BA:7'], ['BA:7', 'BA:8'], ['BA:1'], ['BA:1', 'BA:2'],
      ['SF:'], ['SF:'], ['SF:12327'], ['SF:18608'], ['SF:12476'], ['SF:1001','SF:1002'], 
       ['SF:1002'], ['SF:1001'], ['SF:12475'], ['SF:12475'], ['SF:12477'], ['SF:12477'], 
       ['SF:12478'], ['SF:12478'], ['SF:12479'], ['SF:12479'], 
       ['SF:12476','SF:12475','SF:12477','SF:12478','SF:12479'], 
       ['SF:1002','SF:1001','SF:12475','SF:12477','SF:12478','SF:12479'],
      [':'], [':'], ['SF:','AC:' ,'BA:' ,'CA:' ], ['SF:','AC:' ,'BA:' ,'CA:'], 
      ['AM:','AY:','CC:','CE:','CM:','DE:','EM:','FF:','GG:','HF:','MA:','PE:','RV:', 'SB:','SC:','SL:','SM:','SO:','SR:','ST:','TD:','UC:','VC:','VN:','VT:','WC:','WH:'],
      ['AM:','AY:','CC:','CE:','CM:','DE:','EM:','FF:','GG:','HF:','MA:','PE:','RV:', 'SB:','SC:','SL:','SM:','SO:','SR:','ST:','TD:','UC:','VC:','VN:','VT:','WC:','WH:']]

is_contains = [True, True,
      True, True, True, True,
      True, True, False, False, False, False, False, False, False, False, False, False, 
      True, True, False, False, False, False, 
       False, False, False, False, False, False, 
       False, False, False, False,  
       False,  
       False, 
      True, True, True, True, 
      True,
      True]

Table_names = ['Caltrain', 'Caltrain - TR',
      'Alameda-Contra Costa Transit District (AC Transit)', 'Alameda-Contra Costa Transit District (AC Transit) - TR', '1 AC', '1Tempo AC',
      'BART', 'BART - TR', 'BA Blue', 'BA Blue - TR', 'BA Green', 'BA Green - TR', 'BA Orange', 'BA Orange - TR', 'BA Red', 'BA Red - TR', 'BA Yellow', 'BA Yellow - TR',
      'San Francisco Municipal Transportation Agency (SFMTA)', 'San Francisco Municipal Transportation Agency (SFMTA) - TR', 
               'Line 49 Van Ness/Mission', 'Line 49 Van Ness/Mission - TR', 'Muni Metro K/T', 'Muni Metro K/T - TR', 'Muni Metro K - TR', 'Muni Metro T - TR', 'Muni Metro J', 'Muni Metro J - TR', 'Muni Metro L', 'Muni Metro L - TR', 'Muni Metro M', 'Muni Metro M - TR', 'Muni Metro N', 'Muni Metro N - TR', 
               'Muni Metro', 'Muni Metro - TR',
      'All', 'All - TR', 'BA-CA-SF-AC', 'BA-CA-SF-AC - TR', 'Other', 'Other - TR']
               
print(len(Table_names))              
print(len(rrs))              
print(len(cases))              
print(len(lines))              
print(len(agencies))              
print(len(columns))              
print(len(is_contains))              
Table2 = pd.DataFrame(columns= columns)
i=0        
for agency, line, case, rr, Table_name, is_con in zip (agencies,
                                               lines,
                                               cases,
                                               rrs,
                                               Table_names, 
                                                       is_contains):
    ridership = 0
    for r in rr:
        print(r)
        if is_con:
            if case =='BAU':
                ridership+= RR[RR.index.str.contains(r)]['Baseline'].sum()      
            elif case =='TR':
                ridership+= RR[RR.index.str.contains(r)]['TR'].sum() 
        else:
            if case =='BAU':
                ridership+= RR[RR.index == r]['Baseline'].sum()      
            elif case =='TR':
                ridership+= RR[RR.index == r]['TR'].sum()
        
    Table2.loc[0+i] = [agency, line, # TU
                 case, # TU 
                 int(ridership/Scale),
                int(Table.loc[Table_name, 'Number of Trips per Direction']),
                Table.loc[Table_name, 'VMT km M'],
                Table.loc[Table_name, 'VHT km M'],
                Table.loc[Table_name, 'RPK km M']/Scale,
                Table.loc[Table_name, 'PHT M']/Scale,
                Table.loc[Table_name, 'ASK km M']/Scale,
                Table.loc[Table_name, 'AV Load Factor'],
                Table.loc[Table_name, 'AV Speed vehicles km/h'],
                Table.loc[Table_name, 'AV Speed pass km/h']]
    i+=1

42
42
42
42
42
13
42
CA:
CA:
AC:
AC:
AC:1-142
AC:1T-142
BA:
BA:
BA:11
BA:11
BA:12
BA:5
BA:5
BA:6
BA:3
BA:3
BA:4
BA:7
BA:7
BA:8
BA:1
BA:1
BA:2
SF:
SF:
SF:12327
SF:18608
SF:12476
SF:1001
SF:1002
SF:1002
SF:1001
SF:12475
SF:12475
SF:12477
SF:12477
SF:12478
SF:12478
SF:12479
SF:12479
SF:12476
SF:12475
SF:12477
SF:12478
SF:12479
SF:1002
SF:1001
SF:12475
SF:12477
SF:12478
SF:12479
:
:
SF:
AC:
BA:
CA:
SF:
AC:
BA:
CA:
AM:
AY:
CC:
CE:
CM:
DE:
EM:
FF:
GG:
HF:
MA:
PE:
RV:
SB:
SC:
SL:
SM:
SO:
SR:
ST:
TD:
UC:
VC:
VN:
VT:
WC:
WH:
AM:
AY:
CC:
CE:
CM:
DE:
EM:
FF:
GG:
HF:
MA:
PE:
RV:
SB:
SC:
SL:
SM:
SO:
SR:
ST:
TD:
UC:
VC:
VN:
VT:
WC:
WH:


In [18]:
Table2

,AG,Line,Case,Ridership,# Trips,VMT [M km],VHT [M h],RPK [M km],PHT [M h],ASK [M km],Load Factor,Av Sp. Veh. [km/h],Av Sp. Pas. [km/h]
0,CA,All,BAU,19090,92,7.080731,0.491040,1.073845,0.073931,78.666930,0.013651,14.419866,14.525007
1,CA,All,TR,21280,110,8.416326,0.495227,1.164971,0.067944,93.505383,0.012459,16.994885,17.146086
2,AC,All,BAU,392466,6621,119.850094,17.702160,5.853952,0.817298,26.367021,0.222018,6.770366,7.162568
3,AC,All,TR,390640,6796,122.669050,18.141000,5.629032,0.794242,26.987191,0.208582,6.761978,7.087297
4,AC,1,BAU,14546,239,3.748890,0.778380,0.129981,0.026943,0.824756,0.157599,4.816272,4.824310
5,AC,1T,TR,17116,414,6.597012,1.217220,0.154197,0.029150,1.451343,0.106244,5.419737,5.289690
6,BA,All,BAU,515816,741,46.085164,3.049020,17.167931,1.266460,215.524951,0.079656,15.114746,13.555846
7,BA,All,TR,550366,919,69.574982,4.017180,18.661198,1.161824,325.379000,0.057352,17.319359,16.061986
8,BA,Blue,BAU,98736,141,8.469435,0.535380,2.710504,0.196973,39.608727,0.068432,15.819483,13.760790
9,BA,Blue,TR,97646,180,10.905768,0.615180,2.635774,0.163775,51.002649,0.051679,17.727767,16.093873


# ***Upgrade person file***

In [ ]:
df1 = pd.read_csv('outputs/person_databaseTR-part1.csv')
df2 =  pd.read_csv('outputs/person_databaseTR-part2.csv')
df3 =   pd.read_csv('outputs/person_databaseTR-part3.csv')
print(len(df1))
print(len(df2))
print(len(df3))
# df1 = df1[(df1['Project Tried'] != 'AC - 1TEMPO')&(df1['Project Tried'] != 'SF - Central Subway')]
# df2 = df2[df2['Project Tried']  == 'SF - Central Subway']
# df3 = df3[df3['Project Tried']  == 'AC - 1TEMPO']

df = pd.concat([df1, df2, df3], ignore_index=True)
print(len(df))

df.keys()

In [ ]:
df['Project Tried'].value_counts()

# ***Validation***

In [ ]:
SFMTA = pd.read_excel('inputs/TransitData/SFMTA_2020_01_25.xlsx')

In [ ]:
SFMTA

In [ ]:
SFMTA.keys()

In [ ]:
SFMTA = SFMTA[SFMTA['ROUTE_ALPHA'] == 'KLM']

In [ ]:
list(SFMTA['Stop Name'].value_counts().to_dict())

In [ ]:
SFMTA

In [ ]:
# pd.read_csv('gs://beam-core-outputs/sfbay-baseline-20230526/inexus/sfbay_baseline_default-1.0_2020__20230526.csv.gz')

In [ ]:
person_trips_baseline = pd.read_csv('gs://beam-core-outputs/sfbay-baseline2018-30pct-20230710/inexus/sfbay_baseline_default-1.0_2018__20230709.csv', compression = 'gzip', 
                                    usecols = ['logsum_trip_Potential_INEXUS']) #to Update
person_trips_TR = pd.read_csv('gs://beam-core-outputs/sfbay-tr-30pct-20230716/inexus/sfbay_baseline_default-1.0_2018__20230716.csv', compression = 'gzip', 
                                    usecols = ['logsum_trip_Potential_INEXUS',]) #to Update
# logsum_trip_Potential_INEXUS
# destination_logsum_x
# destination_logsum_y
# logsum_tours_mode_AS_tours
# workplace_location_logsum


In [ ]:
person_trips_baseline.logsum_trip_Potential_INEXUS.sum()

In [ ]:
person_trips_TR.logsum_trip_Potential_INEXUS.sum()

In [ ]:
(1-person_trips_baseline.logsum_trip_Potential_INEXUS.sum()/person_trips_TR.logsum_trip_Potential_INEXUS.sum())


In [ ]:
df.keys()


In [ ]:
(df['Potential INEXUS TR'].sum()-df['Potential INEXUS Baseline'].sum())/df['Potential INEXUS Baseline'].sum()*100


In [ ]:
print(sum([0, 120, 120, 240, 360, 135, 135, 135, 135, 150, 150, 105, 
                 105, 105, 105, 90, 90,90, 90, 120, 120 ,120 ])/60)
print(sum([0, 75, 75, 75, 75, 90, 90, 210, 210, 150, 150, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30])/60)
print(sum([0, 120, 120, 120, 90, 90, 90, 90, 105, 105, 
                105, 105, 150, 150, 135, 135, 135, 135, 360, 240, 120, 120  ])/60)
print(sum([0, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30 , 150 ,150, 210, 210, 90, 90, 75, 75 ,75 ,75 ])/60)

In [ ]:
   "source": [
    "stops_tsb = [7876, 7877, 7878, 7879, 7397, 7359, 7360, 7361, 7362, 7363, 7364, 7365,\n",
    "                7390, 7391, 7392, 7393, 7346, 7344, 7342, 7394, 7395, 7396]\n",
    "tt_stops_tsb = [0, 120, 120, 240, 360, 135, 135, 135, 135, 150, 150, 105, \n",
    "                105, 105, 105, 90, 90,90, 90, 120, 120 ,120 ]\n",
    "\n",
    "stops_ksb = [7217, 6994, 6995, 6997, 6996, 6998, 6991, 6993, 6739, 7058, 7125, 6503,\n",
    "                7114, 5807, 5780, 5786, 5808, 5793, 5798, 5795, 5785, 5418]\n",
    "tt_stops_ksb = [0, 75, 75, 75, 75, 90, 90, 210, 210, 150, 150, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]\n",
    "\n",
    "stops_tnb  = [7398, 7399, 7400, 7347, 7343, 7345, 7401, 7402, 7403, 7404, \n",
    "                 7352, 7353, 7354, 7355, 7356, 7357, 7358, 7166, 7879, 7878, 7877, 7876]\n",
    "tt_stops_tnb = [0, 120, 120, 120, 90, 90, 90, 90, 105, 105, \n",
    "                105, 105, 150, 150, 135, 135, 135, 135, 360, 240, 120, 120 ]\n",
    "\n",
    "stops_knb = [7778, 5784, 5794, 5797, 5787, 5788, 5809, 5779, 5806, 7113, 7109, 6898, \n",
    "                6740, 5730, 5728, 5726, 5419, 5727, 5417, 5731, 6992]\n",
    "tt_stops_knb = [0, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30 , 150 ,150, 210, 210, 90, 90, 75, 75 ,75 ,75]\n",